# Version 2 (sparse matrices...)
### I created the last notebook naively and completely missed that the discusses method operates on sparse matrices, so I'm starting again from scratch

# Generating a sparse matrix representation of a graph...

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_nodes = 100
average_out_degree = 9
total_edge_classes = 36
embedding_size = 128
output_dim = 4



# First we generate some arbitrary graph
def generate_graph(num_nodes, average_out_degree, total_edge_classes):
    num_edges = average_out_degree*num_nodes
    indices = torch.randint(0, num_nodes, (2, num_edges))
    print(indices.size())
    edges = torch.ones(num_edges)
    print(edges.size())
    A = torch.sparse_coo_tensor(indices, edges, (num_nodes, num_nodes))
    return A
# A = generate_graph(15,3,5).to_dense()

# Now maybe it's useful to ensure it's connected...
def generate_connected_graph(num_nodes, average_out_degree, total_edge_classes, embedding_size):
    edges_set ,
    num_edges = average_out_degree * num_nodes

    def sample_random_edge_class():
        return torch.randint(0, total_edge_classes, (1,)).item()
    
    def generate_random_edge():
        u, v = torch.randint(0, num_nodes, (2,)).tolist()
        #no self loops
        while u == v:
            u, v = torch.randint(0, num_nodes, (2,)).tolist()
        return (u, v)
    
    def generate_random_node():
        return torch.randint(0, num_nodes, (1,)).item()
    
    def sample_random_node(A):
        connected_nodes = A._indices()[1].tolist() #select from incoming to...
        if not connected_nodes:
            return generate_random_node()
        return connected_nodes[torch.randint(0, len(connected_nodes), (1,)).item()]#torch.nonzero(A)
    
    initial_edge = generate_random_edge()
    edges_set ,.add(initial_edge)
    indices = torch.tensor([[initial_edge[0]], [initial_edge[1]]], dtype=torch.long, device=device)
    edges = torch.ones(1, device=device)
    A = torch.sparse_coo_tensor(indices, edges, (num_nodes, num_nodes), device=device)
    A_list = []
    for edge_class in range(total_edge_classes):
        for _ in range(num_edges - 1):
            u = sample_random_node(A)
            v = generate_random_node()
            while u == v:
                v = generate_random_node()
            new_edge = (u, v)#torch.cat([u, v], dim=0).unsqueeze(1)
            if new_edge in edges_set ,:
                continue
            edges_set ,.add(new_edge)
            edge_tensor = torch.tensor([[u], [v]], dtype=torch.long, device=device)
            indices = torch.cat([indices, edge_tensor], dim=1)
            edges = torch.ones(indices.size(1), device=device)
            A = torch.sparse_coo_tensor(indices, edges, (num_nodes, num_nodes), device=device)
            # indices = torch.stack((indices,torch.cat([u,v]).unsqueeze(1)),dim=0)
        # print(edges_set ,)
        A_list.append(A)
    random_labels = torch.rand((num_nodes, embedding_size), device=device)
    return A_list, random_labels 

A, X = generate_connected_graph(num_nodes,average_out_degree,total_edge_classes, embedding_size)

# Now per edge type? or randomly assign edges...? maybe multinomial_sample(1/k)^k?

In [ ]:
device

In [ ]:
print(torch.cuda.device_count())

In [ ]:
torch.cuda.current_device

# GCN

In [ ]:
## stopped working after some edits to the graph generation process

In [ ]:
# import torch.nn as nn
# import torch.functional as f

# k_hop = 3

# def normalize_adjacency(A, self=True): # Self-loop doesn't work with R-GCN
#     size = A.size()[0]
#     A = torch.add(torch.eye(size,device=device).to_sparse(), A)
#     degree = torch.sparse.sum(A, dim=1).to_dense()
#     # print(degree.size())
#     d_inv_sqrt = degree.pow(-0.5)
#     D_inv_sqrt = torch.diag(d_inv_sqrt)
#     A = A.to_dense()
#     normalized_A = D_inv_sqrt @ A @ D_inv_sqrt
#     return normalized_A
# A_prime = normalize_adjacency(A)


# class GCNLayer(nn.Module):
#     def __init__(self, x_dim, y_dim):
#         super(GCNLayer, self).__init__()
#         self.x_dim = x_dim
#         self.y_dim = y_dim
#         # self.A_norm = A_norm

#         self.lin = nn.Linear(x_dim, y_dim)

#     def forward(self, A_norm, X):
#         device = next(self.parameters()).device
#         # print(A_norm, X)
#         transformed = self.lin(X)
#         aggregated = torch.matmul(A_norm, transformed)
#         return aggregated


# class GCN(nn.Module):
#     def __init__(self, x_dim, h_dim, y_dim, max_k_hop):
#         super(GCN, self).__init__()
#         self.max_k_hop =max_k_hop
#         self.x_dim = x_dim
#         self.gcns = nn.ModuleList([GCNLayer(x_dim, h_dim) for _ in range(max_k_hop)])
#         self.final_gcn = GCNLayer(x_dim, y_dim) 
#         self.act = nn.ReLU()
#         self.drop = nn.Dropout(p=0.2)
#         # self.norm = nn.LayerNorm(y_dim)
    
#     def forward(self, A, X):
#         # outer layers
#         device = next(self.parameters()).device
#         for i in range(self.max_k_hop):
#             H = self.gcns[i](A, X)
#             H = self.act(H)
#             H = self.drop(H)
#         Y = self.final_gcn(A, H)
#         out = self.act(Y)
#         return out#nn.LogSoftmax(Y)
# model = GCN(X.size(1), embedding_size, output_dim, k_hop).to(device)
# out = model(A_prime, X)


# R-GCN

### Block diagonal weight matrix (one is held in memory for each relational weight per layer, so block diagonal sparse matrices save some memory at the cost of some layer-level information flow)

In [ ]:
block_size = embedding_size//4 #some partition... ensure it's a round number
relation_weights = [torch.randn(block_size, block_size), torch.randn(block_size, block_size)]

block_diag_matrix = torch.block_diag(*relation_weights).to_sparse()
block_diag_matrix.to_dense()

In [ ]:
print( embedding_size//4)

# R-GCN model:

In [ ]:
def normalize_adjacency(A, self=True):
    size = A.size()[0]
    A = torch.add(torch.eye(size,device=device).to_sparse(), A)
    degree = torch.sparse.sum(A, dim=1).to_dense()
    d_inv_sqrt = degree.pow(-0.5)
    D_inv_sqrt = torch.diag(d_inv_sqrt)
    A = A.to_dense()
    normalized_A = D_inv_sqrt @ A @ D_inv_sqrt
    return normalized_A

A_list = [normalize_adjacency(a, False) for a in A]


##save non-error version
def block_diag (weights):
    block_diag_matrix = torch.block_diag(*weights).to_sparse()
    return block_diag_matrix
    

class R_GCNLayer(nn.Module):
    def __init__(self, x_dim, y_dim, num_relations, block_split):
        super(R_GCNLayer, self).__init__()
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.num_relations = num_relations
        gain = nn.init.calculate_gain('relu')
        max_block_size = 1
        for i in range(1,11):
            if x_dim % i == 0 and y_dim % i == 0:
                max_block_size +=1 
        x_block_size = x_dim // block_split
        y_block_size = y_dim // block_split
        self.W = nn.ParameterList()
        for _ in range(num_relations):
            wr = nn.ParameterList()
            for i in range(block_split):
                w = nn.Parameter(torch.randn(x_block_size, y_block_size)) 
                nn.init.kaiming_uniform_(w, a=gain)
                wr.append(w)
            self.W.append(wr)

        self.bias = nn.Parameter(torch.zeros(y_dim))
        nn.init.zeros_(self.bias)

    

    def forward(self, A, X):
        device = next(self.parameters()).device
        aggregated = torch.zeros((X.size(0), self.y_dim), device=device)
        for r in range(self.num_relations):
            weighted = torch.matmul(X, block_diag(self.W[r]))
            transformed = torch.sparse.mm(A[r], weighted)
            aggregated += transformed
        aggregated += self.bias
        return aggregated


class R_GCN(nn.Module):
    def __init__(self, x_dim, h_dim, y_dim, max_k_hop, num_relations):
        super(R_GCN, self).__init__()
        self.max_k_hop =max_k_hop
        self.num_relations = num_relations
        self.x_dim = x_dim
        self.block_split = 2
        self.gcns = nn.ModuleList([R_GCNLayer(x_dim, h_dim, num_relations, self.block_split) for _ in range(max_k_hop)])
        self.final_r_gcn = R_GCNLayer(x_dim, y_dim,num_relations, self.block_split) 
        self.act = nn.ReLU()
        self.drop = nn.Dropout(p=0.2)
        self.norm = nn.BatchNorm1d(y_dim)


    
        
    def forward(self, A, X):
        device = next(self.parameters()).device
        for i in range(self.max_k_hop):
            H = self.gcns[i](A, X)
            H = self.act(H)
            H = self.drop(H)
        Y = self.final_r_gcn(A, H)
        out = self.act(Y)
        out = self.norm(out)
        return out
model = R_GCN(X.size(1), embedding_size, output_dim, k_hop, total_edge_classes).to(device)
out = model(A_list, X)
out

In [ ]:
output_dim

# Train

In [ ]:
from torchviz import make_dot
from tqdm import tqdm
criterion = nn.CrossEntropyLoss()
params = list(model.parameters())
optimizer = torch.optim.Adam(params,lr=0.01)

num_epochs = 600

true_labels = torch.randint(0, output_dim, (num_nodes,), dtype=torch.long).to(device)

def save_gradient_hook(grad):
    gradients.append(grad)


for epoch in tqdm(range(num_epochs)):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    i = 0

    predicted_labels = model(A_list, X)
    
    loss = criterion(predicted_labels, true_labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    i += 1
    epoch_loss += loss.item()

    _, predicted = torch.max(predicted_labels.data, 1)
    total += true_labels.size(0)
    correct += (predicted == true_labels).sum().item()
    avg_loss = epoch_loss
    accuracy = 100 * correct / total
    if epoch % 5 == 0:
        print(f"epoch: {epoch}\navg loss: {avg_loss} accuracy: {accuracy}")
        # make_dot(predicted_labels, params=dict(list(model.named_parameters()))).render("r_gcn_torchviz", format="png")


           

# I'm confused about the rdf stuff. I don't see any multimodal data.

In [ ]:
true_labels

In [ ]:
predicted

# Create graph from n-triple file:

In [1]:
import rdflib
from rdflib import Graph
import logging
import os
from IPython.display import clear_output
import random


data_loc = './Downloads/ml4g/dmg/mmkg/dmg/scripts/dmg777k_stripped.nt'
mini_loc = './Downloads/ml4g/dmg/mmkg/dmg/scripts/dmg777k_stripped_mini.nt'
test_loc = "./Downloads/ml4g/dmg/mmkg/dmg/scripts/dmg777k_test_set.nt"
folder = './Downloads/ml4g'
subsample_chance = 1
def create_mini_nt(infile, outfile):
    with open(infile, 'r', encoding='utf-8') as inp, open(outfile, 'w', encoding='utf-8') as out:
        for i, line in enumerate(inp):
            if random.random() <= subsample_chance:
                out.write(line)



def create_new_graph(path, batch_size = 1000, test=True):
    logging.basicConfig(
    filename='rdf_parsing_errors.log',
    filemode='w')
    
    graph = Graph()
    batch_num = 0
    i = 0
    with open(path, 'r', encoding='utf-8') as f:
        while True:
            batch = []
            try:
                
                [batch.append(next(f)) for j in range(batch_size)]
                i += j
            except:
                pass
            if not batch:
                break
            batch_num += 1
            nt_string = ''.join(batch)
            try:
                graph.parse(data=nt_string, format='nt')
                if test:
                    graph = Graph()
            except ParseError as e:
                logging.error(f"in batch: {batch_num}:\npproblematic data:\n\n{batch}\n\n")
                check(batch, batch_num, test=test)

    return graph

def check(batch, batch_num, test = True):
    graph = Graph()
    for i, line in enumerate(batch):
        try:
            graph.parse(line)
        except Exception as e:
            logging.error(f'in line: {i}:\n{line}\n{e}')
graph=create_new_graph(data_loc, test=False)


In [36]:
i=0
unique_classes = set()
unique_predicates = set()
for s,p,o in graph2:
    unique_predicates.add(p)
    if i< 35:
        # print(s,p,o)
        pass
    unique_classes.add(o)
unique_classes

{rdflib.term.URIRef('https://data.labs.pdok.nl/rce/def/agrarischGebouw'),
 rdflib.term.URIRef('https://data.labs.pdok.nl/rce/def/kasteelLandhuis'),
 rdflib.term.URIRef('https://data.labs.pdok.nl/rce/def/kerk'),
 rdflib.term.URIRef('https://data.labs.pdok.nl/rce/def/molen'),
 rdflib.term.URIRef('https://data.labs.pdok.nl/rce/def/woonhuis')}

In [37]:
unique_predicates

{rdflib.term.URIRef('https://data.labs.pdok.nl/rce/def/monumentCode')}

In [31]:
edge_set

{'http://data.pdok.nl/def/pdok#asWKT-RD',
 'http://dbpedia.org/ontology/city',
 'http://dbpedia.org/ontology/codeNationalMonument',
 'http://dbpedia.org/ontology/location',
 'http://dbpedia.org/ontology/name',
 'http://dbpedia.org/ontology/neighbourhood',
 'http://dbpedia.org/ontology/thumbnail',
 'http://purl.org/dc/terms/created',
 'http://purl.org/dc/terms/creator',
 'http://purl.org/dc/terms/description',
 'http://purl.org/dc/terms/isPartOf',
 'http://purl.org/dc/terms/spatial',
 'http://schema.org/dateCreated',
 'http://schema.org/roleName',
 'http://www.geonames.org/ontology#alternateName',
 'http://www.geonames.org/ontology#countryCode',
 'http://www.geonames.org/ontology#featureClass',
 'http://www.geonames.org/ontology#featureCode',
 'http://www.geonames.org/ontology#locationMap',
 'http://www.geonames.org/ontology#name',
 'http://www.geonames.org/ontology#nearbyFeatures',
 'http://www.geonames.org/ontology#officialName',
 'http://www.geonames.org/ontology#parentADM1',
 'http:

In [2]:
from rdflib import Graph
import random
def to_connected_small(g,subsample_chance=0.8, barrier=100,max_len=5000):#25000):
    obj_set = set()
    mini_graph = Graph()
    i=0
    j=0
    for s,p,o in g:
        if j > max_len:
            break
        i+=1
        if random.random() >= subsample_chance and (i<barrier or s in obj_set or o in obj_set):
            obj_set.add(s)
            obj_set.add(o)
            j+=1
            mini_graph.add(triple=(s,p,o))
    print(f"{i}-->{j}")
    return mini_graph
graph = to_connected_small(graph)

114548-->5001


In [12]:
i=0
for s,p,o in graph:
    if i < 20:
        print(s,p,o)

https://data.labs.pdok.nl/rce/id/image/20383170 http://purl.org/dc/terms/creator Technau, Sergé
https://data.labs.pdok.nl/rce/id/image/20475643 http://purl.org/dc/terms/isPartOf Agrarisch Erfgoed - Fotocollectie SHBO
https://data.labs.pdok.nl/rce/id/image/20454446 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://data.labs.pdok.nl/rce/def/Afbeelding
https://data.labs.pdok.nl/rce/id/image/20506974 http://purl.org/dc/terms/isPartOf Agrarisch Erfgoed - Fotocollectie SHBO
http://bag.basisregistraties.overheid.nl/bag/id/geometry/91B21AAC9E9D3F33C7BB85521AC15B84 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.opengis.net/ont/geosparql#Geometry
http://bag.basisregistraties.overheid.nl/bag/id/geometry/57699D1F772A09CA05F8CEC75607E797 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.opengis.net/ont/geosparql#Geometry
https://data.labs.pdok.nl/rce/id/image/20542911 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://data.labs.pdok.nl/rce/def/Afbeelding
http://bag.b

NameError: name 'graph2' is not defined

In [26]:
i

777124

In [25]:
len(edge_set2)

60

In [ ]:
# import pickle

# pickle_file_path = os.path.join(folder, 'rdf_graph_connected.pkl')
# with open(pickle_file_path, 'wb') as f:
#     pickle.dump(graph, f)

In [ ]:
# %%capture
# import pickle
# import os
# import rdflib
# import logging
# folder = './Downloads/ml4g'

# pickle_file_path = os.path.join(folder, 'rdf_graph.pkl')

# with open(pickle_file_path, 'rb') as f:
#     graph = pickle.load(f)

In [36]:
from datetime import datetime
from typing import defaultdict
node_set = set()
http_set_s = set()
http_set_m = set()
http_set_l = set()

edge_set = set()
string_set_s = set()
string_set_m = set()
string_set_l = set()

image_set = set()
num_set = set()
poly_set = set()
datetime_set = set()
date_set = set()
year_set = set()
point_set = set()

text_edge_set_s = set()
text_edge_set_m = set()
text_edge_set_l = set()

image_edge_set = set()
num_edge_set = set()
spatial_edge_set = set()
temporal_edge_set = set()
encoder_map = defaultdict(list)

i = 0

dtype_set = set()

def is_date(date_string):
    try:
        rdflib.term.parse_datetime(date_string)
        return 'datetime'
    except ValueError:
        try:
            rdflib.term.parse_xsd_gyear(date_string)
            return 'year'
        except:
            try:
                rdflib.term.parse_xsd_date(date_string)
                return 'date'
            except:
                return False

def add_str_to_set(string,edge):
    if len(string) < 20:
        string_set_s.add(string)
        text_edge_set_s.add(edge)
    elif len(string) < 50:
        string_set_m.add(string)
        text_edge_set_m.add(edge)
    else:
        string_set_l.add(string)
        text_edge_set_l.add(edge)

for s,p,o in graph:
    i+=1
    pi = p.identifier
    if s.identifier in subj_set:
        print('.')
    for node in [s,o]:
        ni = node.identifier
        node_set.add(ni)
        try:
            dtype = node.datatype.identifier
            dtype_set.add(dtype)
        except AttributeError:
            dtype = ''
        if 'http' in ni[:200]: #200, because images sometimes have kgbench url attached
            if len(ni) < 20:
                http_set_s.add(''.join(ni.split(':')[1].split('/')[:-1]))
            elif len(ni) < 50:
                http_set_m.add(''.join(ni.split(':')[1].split('/')[:-1]))
            else:
                http_set_l.add(''.join(ni.split(':')[1].split('/')[:-1]))

        else:
            if is_date(ni) and ('Year' in dtype or 'date' in dtype):
                date_type = is_date(ni)
                if date_type == 'datetime':
                    datetime_set.add(ni)
                elif date_type == 'year':
                    year_set.add(ni)
                elif date_type == 'date':
                    date_set.add(ni)
                    
            elif node.isalnum():
                if node.isnumeric():
                    if node.isdigit():
                        num_set.add(int(node.identifier))
                        num_edge_set.add(pi)
                    else:
                        num_set.add(float(node.identifier))
            elif ni.startswith('POINT') or ni.startswith('Point'): #didn't see any points, but according to the paper they can be included.
                point_set.add(ni)
                spatial_edge_set.add(pi)
            elif node.isalpha(): #maybe elif maybe not dunno if it filters out strings with numbers
                add_str_to_set(ni.pi)
            elif ni.startswith('_9j_'):
                image_set.add(ni) #might want to load this to hard drive if memory becomes an issue.
                image_edge_set.add(pi)
            elif ni.startswith('POLYGON') or ni.startswith('Polygon'):
                poly_set.add(ni)
                spatial_edge_set.add(pi)                
                temporal_edge_set.add(pi)
            elif ni.lower().startswith('multipolygon'):
                poly_set.add(ni)
            elif ni.isascii():
                add_str_to_set(ni,pi)
            elif ni.isprintable():
                add_str_to_set(ascii(ni),pi) #don't know if it's necessary, but it probably can't hurt
            else:
                add_str_to_set(ascii(ni),pi)

                

    edge_set.add(pi)

    
i

.
.
.
.
.
.
.
.
.
.
.
.


5001

In [35]:
subj_set = set()
for s,p,o in graph2:
    subj_set.add(s.identifier)
subj_set

{'https://data.labs.pdok.nl/rce/id/monument/513188',
 'https://data.labs.pdok.nl/rce/id/monument/40595',
 'https://data.labs.pdok.nl/rce/id/monument/39113',
 'https://data.labs.pdok.nl/rce/id/monument/521640',
 'https://data.labs.pdok.nl/rce/id/monument/37921',
 'https://data.labs.pdok.nl/rce/id/monument/386790',
 'https://data.labs.pdok.nl/rce/id/monument/522081',
 'https://data.labs.pdok.nl/rce/id/monument/521155',
 'https://data.labs.pdok.nl/rce/id/monument/486758',
 'https://data.labs.pdok.nl/rce/id/monument/39079',
 'https://data.labs.pdok.nl/rce/id/monument/40643',
 'https://data.labs.pdok.nl/rce/id/monument/516981',
 'https://data.labs.pdok.nl/rce/id/monument/520521',
 'https://data.labs.pdok.nl/rce/id/monument/37838',
 'https://data.labs.pdok.nl/rce/id/monument/527674',
 'https://data.labs.pdok.nl/rce/id/monument/515511',
 'https://data.labs.pdok.nl/rce/id/monument/514013',
 'https://data.labs.pdok.nl/rce/id/monument/524373',
 'https://data.labs.pdok.nl/rce/id/monument/521186',

In [4]:
import torchvision.transforms as trv


transform_temp = trv.Compose([
    trv.Resize((224, 224)),
    trv.ToTensor(), trv.Normalize(
        mean=[0.5, 0.5, 0.5], 
        std=[0.3, 0.3, 0.3])]) 

In [5]:
import math
import torch
from torch.utils.data import Dataset, DataLoader
from string import printable
import pickle


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

node_map = {i:node for i,node in enumerate(node_set)}
inv_node_map = {node:i for i,node in enumerate(node_set)}

edge_map = {i:edge for i,edge in enumerate(edge_set)}
inv_edge_map = {edge:i for i,edge in enumerate(edge_set)}


def tokenize_string(s, character_map, max_len=5000):
    s = s[:max_len]
    tokens = [character_map[char] for char in s]
    return tokens #

def encode_image(encoded_str, transform):
    #preprocess for encoder[2](2 layer cnn -->embedding_dim)
    image, img_bytes = decode_base64_image(encoded_str)
    return transform(image), img_bytes

def encode_num(n, max_num, log=False):
    v = n/max_num
    v = math.log(v) if log else v
    return torch.tensor((v),dtype=torch.float32, device=device)

def encode_polygon(poly, global_mean_x, global_mean_y, x_max, y_max):
    poly_tensor_x = (global_mean_x-torch.tensor([ point[0] for point in poly], 
                                                dtype=torch.float32, device=device))/x_max
    poly_tensor_y = (global_mean_y-torch.tensor([ point[1] for point in poly], 
                                                dtype=torch.float32, device=device))/y_max
    return torch.stack((poly_tensor_x,poly_tensor_y),dim=0)

def encode_point(point, max_x, max_y):
    
    point = torch.tensor(point,dtype=torch.float32, device=device)
    div = torch.tensor((max_x,max_y),dtype=torch.float32, device=device)
    return point/div

def encode_date(date):
    def cyclical(num, max_num, epsilon = 1e-8):
        # cyclical: [sine((2pi * X)/max_num_of_cycle) cos((2pi * X)/max_num_of_cycle)]
        return torch.tensor([math.sin((2 * math.pi * num)/max_num)+epsilon, 
                             math.cos((2 * math.pi * num)/max_num)+epsilon],
                            dtype=torch.float32, device=device)
        
    def norm_cent(num):
        # non-cyclical only centuries: normalized from -99 to 99 (-9999 bc to 9999 ac)
        return torch.tensor((num + 99)/198,dtype=torch.float32, device=device).unsqueeze(0)
        
    split_str = date.split('-')
    years_str = split_str[0]
    month_str = split_str[1]
    day_str = split_str[2]
    centuries = norm_cent(int(years_str[:-2]))
    decades = cyclical(int(years_str[-2]), 10)
    years = cyclical(int(years_str[-1]), 10)
    months = cyclical(int(month_str), 12)
    days = cyclical(int(day_str), 31)
    return torch.cat((centuries, decades, years, months, days), dim=0)

def encode_year(year):
    #preprocess for temporal encoder[4](? layer ffnn -->embedding_dim)
    def cyclical(num, max_num, epsilon = 1e-8):
        # cyclical: [sine((2pi * X)/max_num_of_cycle) cos((2pi * X)/max_num_of_cycle)]
        return torch.tensor([math.sin((2 * math.pi * num)/max_num)+epsilon, 
                             math.cos((2 * math.pi * num)/max_num)+epsilon],
                            dtype=torch.float32, device=device)
        
    def norm_cent(num):
        # non-cyclical only centuries: normalized from -99 to 99 (-9999 bc to 9999 ac)
        return torch.tensor((num + 99)/198,dtype=torch.float32, device=device).unsqueeze(0)
    years_str = year
    centuries = norm_cent(int(years_str[:-2]))
    decades = cyclical(int(years_str[-2]), 10)
    years = cyclical(int(years_str[-1]), 10)
    return torch.cat((centuries, decades, years), dim=0)


class TextDataset(Dataset):
    def __init__(self, strings, character_map, max_length=5000):
        # self.strings = strings
        self.character_map = character_map
        self.max_length = max_length
        self.strings = strings
        self.ids = []
        self.pad = False
        ids = []
        for string in strings:
            try:
                self.ids.append(inv_node_map[string])
            except KeyError:
                self.ids.append(-1) #

    def __len__(self):
        return len(self.strings)

    def __getitem__(self, i, pad=True):
        s = self.strings[i]
        tokens = tokenize_string(s, self.character_map, max_len=self.max_length)

        return torch.tensor([self.ids[i]]+tokens, dtype=torch.long, device=device)


class ImageDataset(Dataset):
    def __init__(self, encoded_images, transform):
        self.encoded_images = encoded_images
        self.transform = transform
        self.ids = [inv_node_map[encoded_image] for encoded_image in encoded_images]

    def __len__(self):
        return len(self.encoded_images)

    def __getitem__(self, i):
        encoded_str = self.encoded_images[i]
        image, _ = decode_base64_image(encoded_str, log_note=f"inside generator, image nr: {i}")
        image = self.transform(image)
        return self.ids[i], image

class NumericalDataset(Dataset):
    def __init__(self, numbers, log_scale=False):
        self.numbers = numbers
        self.log_scale = log_scale
        self.max_num = max(numbers)
        self.ids = [inv_node_map[str(number)] for number in numbers]

    def __len__(self):
        return len(self.numbers)

    def __getitem__(self, i):
        n = self.numbers[i]
        norm = n / self.max_num
        norm = math.log(norm + 1e-8) if self.log_scale else norm
        return self.ids[i], torch.tensor([norm], dtype=torch.float32, device=device)

class SpatialDataset(Dataset):
    def __init__(self, spatial_data, global_mean_x, global_mean_y, x_max, y_max, ids):
        self.spatial_data = spatial_data
        self.global_mean_x = global_mean_x
        self.global_mean_y = global_mean_y
        self.x_max = x_max
        self.y_max = y_max
        self.max_len = max([len(data) for data in spatial_data])
        self.ids = ids#[inv_node_map[spatial_datum] for spatial_datum in spatial_data]

    def __len__(self):
        return len(self.spatial_data)

    def __getitem__(self, i):
        spatial = self.spatial_data[i]
        spatial_tensors = []
        if isinstance(spatial, list):
            for x,y,a,b,c in spatial:
                
                x = (x - self.global_mean_x) / self.x_max
                y = (y - self.global_mean_y) / self.y_max
                coord_tensor = torch.tensor([x,y,a,b,c], device=device)
                spatial_tensors.append(coord_tensor)
            spatial_tensor = torch.stack(spatial_tensors,dim=0)
        elif isinstance(spatial, tuple):
            x, y,_,_,_ = spatial
            spatial_tensor = torch.tensor([(x / self.x_max, y / self.y_max,0,0,0)], dtype=torch.float32, device=device)
        else:
            spatial_tensor = torch.zeros((1, 2), dtype=torch.float32, device=device)
        return self.ids[i], spatial_tensor


class DateTimeDataset(Dataset):
    # todo: check if there are other temporal feature types in the data. i.e. just years etc.
    def __init__(self, dates):
        
        self.dates = dates
        self.ids = [inv_node_map[date] for date in dates]


    def __len__(self):
        return len(self.dates)

    def __getitem__(self, i):
        date = self.dates[i]
        date_feature = encode_date(date)
        return self.ids[i], date_feature

class YearDataset(Dataset):
    def __init__(self, dates):
        
        self.dates = dates
        self.ids = [inv_node_map[date] for date in dates]

    def __len__(self):
        return len(self.dates)

    def __getitem__(self, i):
        date = self.dates[i]
        date_feature = encode_year(date)
        return self.ids[i], date_feature

def process_point(point_str, highest_x, highest_y):
    if 'POINT' in point_str:
        point_str = point_str.split('POINT(')[1].split('))')[0]
    elif 'Point' in point_str:
        point_str = point_str.split('Point(')[1].split('))')[0]
    point_list = point_str.strip(')').strip('(').split()
    point = [float(coord) for coord in point_list]
    point.extend([0,0,0])
    point = tuple(point)
    
    point_x,point_y,_,_,_ = point
    highest_x = point_x if point_x > highest_x else highest_x
    highest_y = point_y if point_y > highest_y else highest_y
    return point, highest_x, highest_y

    
def process_multipoly(multipoly_str, max_x, max_y):
    multipoly_str = multipoly_str.split('(((')[-1].split(')))')[0]
    sub_polys = multipoly_str.split(') (')
    multipoly = []
    for i, sub_poly in enumerate(sub_polys):
        aligned_sub_poly = 'POLYGON ((' + sub_poly + '))'
        multipoly.append(get_num_data(aligned_sub_poly, max_x, max_y, last_multi=i==len(sub_polys)))
    return multipoly
        

def get_num_data(poly_str, max_x, y_max, last_multi=False):
    poly_str = poly_str.split('(')[-1].split(')')[0]
    poly_combi_str_list = [poly for poly in poly_str.split(',')]
    try:
        poly_tupled = [(float(poly.split()[0]),float(poly.split()[1]),1,0,0) if i < len(poly_combi_str_list)-1  else (float(poly.split()[0]),float(poly.split()[1]),0,0,1) if last_multi else (float(poly.split()[0]),float(poly.split()[1]),0,1,0)
                       for i,poly in enumerate(poly_combi_str_list)]
    except ValueError:
        poly_tupled = [(float(poly.strip(')').strip('(').split()[0]),float(poly.strip(')').strip('(').split()[1]),1,0,0) if i < len(poly_combi_str_list)-1  else (float(poly.split()[0]),float(poly.split()[1]),0,0,1) if last_multi else (float(poly.split()[0]),float(poly.split()[1]),0,0,1)
                       for i,poly in enumerate(poly_combi_str_list)]
    x_max, y_max = max([x for x,y,_,_,_ in poly_tupled]), max([y for x, y,_,_,_ in poly_tupled])

    x_max =  x_max if x_max > max_x else max_x
    y_max =  y_max if y_max > max_y else max_y
    x_mean = sum([tup[0] for tup in poly_tupled])/len(poly_tupled)
    y_mean = sum([tup[1] for tup in poly_tupled])/len(poly_tupled)
    return poly_tupled, x_mean, y_mean, x_max, y_max

global_mean_x = 0
global_mean_y = 0

max_x = 0
max_y = 0

node_map = {i: node for i, node in enumerate(node_set)}
inv_node_map = {node: i for i, node in enumerate(node_set)}

points_tupled = []
point_ids = []
for i, point in enumerate(point_set):
    point_tupled,max_x,max_y = process_point(point,max_x,max_y)
    points_tupled.append(point_tupled)
    point_ids.append(inv_node_map[point])


polys_tupled = []
poly_ids = []
x_max, y_max = 0,0
done = False
multipolys_tupled = []
for i, poly in enumerate(poly_set):
    if 'multi' in poly.lower():
        multipolys_tupled.append(process_multipoly(poly, x_max, y_max))
        
    poly_tupled, x_mean, y_mean, x_max, y_max = get_num_data(poly, 
                                                             x_max, y_max)
    global_mean_x += 1
    global_mean_y += 1
    polys_tupled.append(poly_tupled)
    poly_ids.append(inv_node_map[poly])
i += 1
# print(poly_tupled[0])
global_mean_x, global_mean_y = global_mean_x/i, global_mean_y/i


character_map = {char:i for i,char in enumerate(printable)}
character_map['\x7f'] = 101

# node_set.update(string_set_s)
# node_set.update(string_set_m)
# node_set.update(string_set_l)

text_dataset_s = TextDataset(list(string_set_s), character_map)
text_dataset_m = TextDataset(list(string_set_m), character_map)
text_dataset_l = TextDataset(list(string_set_l), character_map)

image_dataset = ImageDataset(list(image_set), transform_temp)

numerical_dataset = NumericalDataset(list(num_set), log_scale=True)

spatial_dataset = SpatialDataset(polys_tupled + points_tupled, global_mean_x, global_mean_y, x_max, y_max, ids=point_ids + poly_ids)

datetime_dataset = DateTimeDataset(list(datetime_set)) #if possible combine
year_dataset = YearDataset(list(year_set))

/home/royal-cookings/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


# Grab train, valid, test sets

In [53]:
test_label_set = set()
train_label_set = set()
val_label_set = set()


test_node_label_map = defaultdict(list)
train_node_label_map = defaultdict(list)
val_node_label_map = defaultdict(list)


test_loc = "./Downloads/ml4g/dmg/mmkg/dmg/scripts/dmg777k_test_set.nt"
train_loc = "./Downloads/ml4g/dmg/mmkg/dmg/scripts/dmg777k_train_set.nt"
val_loc = "./Downloads/ml4g/dmg/mmkg/dmg/scripts/dmg777k_valid_set.nt"

graph_test = create_new_graph(test_loc, test=False)
graph_train = create_new_graph(train_loc, test=False)
graph_val = create_new_graph(val_loc, test=False)

for s,p,o in graph_test:
    test_label_set.add(o.identifier)
    try:
        node_id = inv_node_map[s.identifier]
        test_node_label_map[node_id] = o.identifier
    except: #not everything has to be in node_set with mini graph
        pass
for s,p,o in graph_train:
    train_label_set.add(o.identifier)
    try:
        node_id = inv_node_map[s.identifier]
        train_node_label_map[node_id] = o.identifier
    except: #not everything has to be in node_set with mini graph
        pass

for s,p,o in graph_val:
    val_label_set.add(o.identifier)
    try:
        node_id = inv_node_map[s.identifier]
        val_node_label_map[node_id] = o.identifier
    except: #not everything has to be in node_set with mini graph
        pass



defaultdict(list,
            {201: 'https://data.labs.pdok.nl/rce/def/woonhuis',
             3482: 'https://data.labs.pdok.nl/rce/def/kerk',
             480: 'https://data.labs.pdok.nl/rce/def/kasteelLandhuis',
             721: 'https://data.labs.pdok.nl/rce/def/molen',
             3871: 'https://data.labs.pdok.nl/rce/def/molen',
             3724: 'https://data.labs.pdok.nl/rce/def/woonhuis',
             3228: 'https://data.labs.pdok.nl/rce/def/kerk',
             2464: 'https://data.labs.pdok.nl/rce/def/woonhuis',
             4238: 'https://data.labs.pdok.nl/rce/def/agrarischGebouw',
             2985: 'https://data.labs.pdok.nl/rce/def/kasteelLandhuis',
             2640: 'https://data.labs.pdok.nl/rce/def/agrarischGebouw',
             2441: 'https://data.labs.pdok.nl/rce/def/agrarischGebouw',
             2756: 'https://data.labs.pdok.nl/rce/def/woonhuis',
             1731: 'https://data.labs.pdok.nl/rce/def/woonhuis',
             2439: 'https://data.labs.pdok.nl/rce/def/

In [56]:
import base64
from PIL import Image
from io import BytesIO

byte_str = """_9j_4AAQSkZJRgABAQAAAQABAAD_2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL_2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL_wAARCADIAMYDASIAAhEBAxEB_8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL_8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4-Tl5ufo6erx8vP09fb3-Pn6_8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL_8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3-Pn6_9oADAMBAAIRAxEAPwCprVoYUDmLGeoYjIA9hWBHbLOwCRlSGBJPrXea6kQtGYtiQ8Llep5zgGuXdLxpIwwGwbQ2ByR_n-dAHuelqPsUYZgW2jJ7nAq2QAQTwSKpacFFlGTkcA8_SrDNkjJyBmgCpdOoYnOSR2NY05XyJGOSCMDPrV-4fbkBSQQeO5rJuZWNm25SCG5HoKAPPvF00kFhIyShCXA4GSfb2HSuStNd1C3Kxm5MqZIYE5yD1Fdb4vgFzp7omWYygqAOSP6VwLWDxZAjJweSD0oA7Ky1xL6QCX5cgABiARgngH0res9TidnCbgFIBU8ZA64_GvKBJNbSBgoGCRhhkHn-VdBpniWBGVZ0EJGASASPrxzQB61azpLaI0irJGOAc4I9jj0p1xZJMQNxKgcIQRg8etcrpXiSyRZJSzJGynCA7txJx2zg8-oq_wD8JLbIzBTK4BG1wMjp69aAJLiyltc4BZTgYx07ZH481C4LEA8kgEEGtq31KKeMAYdmAyufu57ntVTUEBuA6qFUAnBA4PtigCmHBBCtgAgDIOcYwMfrUlvKDqAXAII2hSCSMY5yDUTqyKDgHcwOPTNOtCZZyxKoRnGBjOBg9-en60Aa0wICFAMAEnH-fapLZmcgkAYAHOe57flUaxBII95IYgkHcDk5B59OaW2YNcADKgEnk-h60AOmJNsGOWJzk9-uetcdrj7JyMndnccDggdK7PG5QoAA5GB0_wA81yevRIt0wODwcd8UAGm3oRyobI8ssQTnJ_zj8qivEMkiYYH5QcgAc4GR_n0rKtWaEs4LbwflIIBAJx_WtX5nRmAYkLgcYPAwcfiD-dAE-naY76hbEszRZ3tkcE4P5V1kduiXKFUUHBIwQNvXqa57RbgGOMHJG3kknAORgc9_8K1Uu5FuRIylQRgnA4OB3H4UAdCoZgUds4OT06_5NFZ0d-H3bNxxjIooAzL-3kn8lQqyEsCy4Bz659O3NcncQONRk5JjVwAFOCef5cela_8AwkFoQWt5ltiyhSuAcnjJPt7VWs7yB7gKLhZVZsvgAsR2Oeg7UAer2ZBtVXp8ox-VTFCTkjGPf2pLJM24YDGQB9OKlQKM7iSTnmgDLuBtBLcnnGKxZ2L20gALAHJFbl8QvBxxnk1izyKljIWcBC2Dgc0AcNrYLxNgEAMBgEA_XmuUFtOGkxMucZIYZGOeP89q63UozcbgoPUYOcAjOP51jX2nsqloywjZcYAxzwPrgmgDmJ8u7EwxkcE8cD6f_XrPn2rlRCpkJyACQAPfNdNbwJEHViTIAcjsD2P5fzqr_ZrvJ5rKWDHO4nt1HXmgDHtoNRQh4Qy5BJUcD8a7HQor29CG6XYgx0AJPbNX9O0xBEiqoGBnGOhPbPX8637OzEAUbQQDk57DigC5bW8VtDtyST6HI6UStvUDIyTkA8ggdKmyHIXJKjoM85qKQA7ARgAkD86AKhGXJfHyYOMc9-1Fkn-kKpcE4J4OM8_T3qaQHLYOGxgEZOf85NNjQxXEbksSCSBwSBQBo7chVA4UnJI6f_XqRIsThy4y3I54HbH54pzozRgHAUHJA5yfT-tMHDAZJAwcnvzQBHI5QMSCcA4zxk1wuoXbvdNwc9eDxjH-NdxOn3hwck8Z6Vw2rWrrcbzjBzyB6mgBkE0krBlKhVbGfUn1_KtyyjJt7gKATjjJwDyelYWnKoLAkABgQCePX866nSlIt5CVAzntwOucdaAOft9WFuk0E2NhIAOf0_L-VbI1RbqERKoIGQcnkDoffNZlzpyT3JthuxI4w6kgryDjHfua2G0qOwmjRGIBTcWducnqMeg5NAE1o-IF-c7sc7TtI-vBopthG8YO4qAckHkcZ_H_ACKKAPLriweQhrS9iJCgnLgZP0NWbeW6tLu2AuFk3SqCAMHJI7irN14ahQhopmjJzwRjAziorLRNShvrdo5VeMTLnAzxkelAH0fZA_YQAcEgEflSsCGGBnA6VPbbBahRxwD-lNc4J2gE0AZU4ad2DAAEYGayLu3AtGBGRuzjGefpW1dy-VC77SccgDrWPJKstmxUkFjkjuPagDidUPkAnIBzxkkj6_8A1vWs4XaSRRqZA5IJIzgD_wCv9PWpPGtwYLcusY2BwMEnuPauStNWuVDMkWRjHCk9e59DyeaAOgjgtGuCxMjEEq2D1HYZ9aswWk8sgiYkorDGeSBwM-9Q6FCb2RNwdCSNzEYU8dgev1roruJrNS6qpjYAZBGRxnnvQBHDETIqIxIB6AEYx-H1rSm2IwYtsA55_wA9DVe2urRCzLMuABg8c-9aA1XS1jBleDkDO4D8snqOOg6UAMidHUMpBBOMgfnTZcBeSc5yABVK78R6NEpAmAkxkBFIB9sHj16d6gTxLpM8YZ5wpBGBtJIx1J_L3oAunCyKWHGM5zzj0qzbIjT5JAAHUnr16EcfyrBfxJYvIxaUshBw4UYB9wKINatyGf7UUyAOAcMO-ff8aAOtX94MA4Ge3YAdB2obbwrEqCQRz29KwV8TafF5cTSFVB3b8_eH8s8frVhNZ0y6mVVvFQE5JckD_P6UAajRCQkEdSc_zrE1TTRKEKKFAJJweuMf4Vt27xyoxilVweFZSMNzjP6U6RGdSuSMjBxQBw6WgEgVVJDNk7TyQSAfrXQ2YP2aRQAQHIx0yeMfTtUMtuttIrlc4JA-mc_jVuBBBb5YnJYsOSOgx0oAoFFGtWyPlQ7EjAyc44wPwH4Vf1uymub6NAFCqocYGSQc5HH41Lp0cLX7zsrb1UnLjhSDjIz35rUkNtLKJDPGqBSFIccj1x6f40AYltMqWy-ZIFUcBgaK6SCOzWBX8xNrc7lIBJ_PpRQB5VqEzSlsNkkAZPYDpUOnorXtv5mQ3mKARxxkVau1XzioAJPGe461Lp8KfaoAACxkXPHTkdKAPa1GYwqnAAA-tWpAI4wAAKhjQJGCMYwOnrSPMWG1h0PBoAzZwSzK2cAE1z8rgI4AIBY4OO1dHclRLIpODt4Getc1cYS2dt2FBJOfSgDifEkDX6GCJwp8wEE9wO3esCDw46c3V6ozwQpyRn8a7DUY1eFGiBVg3XHXOMACkGnLcG2ntUMwEoaVGYKAARwc9e3NAFeDwJaqimeeQqD124BGO-DmnHwJYOw8u-ZI2OAGGST6A556V03203NvMFAQo-0FwMHHfmqxhUoAQTj5sEcAjg4_P9KAMaLwLYzBdt6mSeSYjg_gDxXFap4A8Tf8JC9vBqKG1yGV1kICqTgHaT146Z5PevXLWCRIgzgjJwATz-P4VBMxGvi3U4Row7HHoCcfSgDkLH4XQxkNqGt3N2cYBWMKB-BJz-NaD_C_w9LIHW5vwx4-SRVH5BcV2JQRgEZzjgdv8iobncYSqAgjGUI9PTvQByjfCrRCcLc3wToQsqjJz3G3rU5-GWhNOHie7j2oAUMoZTjjJBGcn2NdGZ1ghBLgA4GSDnJ4xjFSoMxl5M4PAwefxoA5-P4daa4wLmUDoVxkfzqMfDCyG4rfTD0BX_69dZbzqkUjg7huIwDz_npUH2p9rEsSxPHPSgDC0zwuNOMkUWo3AIblSAFyeM9xUkr6vbTlZ4Z5yACro-VYZPAAAAPscVrQvvkk3Pk5HfHGBV6aQCNMHr69DQBw95PrETq4sBKgbJjBKsD9STn8qq3Or-Jb0quk-GCrIAGmuJQFUkc4wRmvQTtBQ4BBOMEdsdRVC9uGhuAUyAy5II47frQByg8MeJtSEY1PxDHCCc-XFESFPcA5HPH6VcT4eCQBLnxJfsDztQAD8yTWnHqEZwC-9xIMKR1GOxJ9-aspeIhZi6tjg44yMDjAzjigCrB8N9IjT97eX0oPTdKAf0FFbUV-HiDJgg_3T_nvmigDy-WTzDuAGCeuK19JjUGKYgMWdQMgYGCOayo1IlZwgKrnI64Jqzp-rG3vhbvEDG0q4GeQSQOD_SgD2UuyxgYyCB0qKUuHO0fU1dRR5CMQAcDpUEhIZjkdOM0AY-oiVpiFIAx1z1rn5gXt5Y2GcHJz39s1vX0reYflBBBHT2rGlRZLVwFIyCRkfrQBgy4EpgdwiPgnknAGD2pNLuUUtbAqxWQggk8qec4P1pFHnapBEMqCwBIGTkketaP9mwW801wmOUCjB6kHv-A_SgBt5P5csazkBpQAY15x6HPvj-VWYYgzqwbCAYCEDj2PtWPqSQ31xHHvKoWUA9SOfXj863Ft_L2YOQDjr79eaALsmSEA7Hg1HDaB_FqSFSwMPzDB44AAz78_lVTxB4gsvCmjm-uwJpHGIYRwWP8AQV5HFr_jrxffzXOlzPaQ_dLRHy0AHQZOSSM__qoA9-lEMShWBJYbcAZ59M-tY98EjkVULgHklxk88YyK8ug1_wCIXht0k1J11azBy8bkMwHcggZBx3Oa9JtNVsPFHhyHU9McgD5JEcYZGGMqR2I-voe9AFc3RRmLIWC5wPX3_Ko7fWXM7RtCTAYwxQnlST9fcH2p0iskTyl8KoJJGMgen1zx-Nc5bKZL5353bCRg9QD0OPb-VAHZhgbePa2QcnGeuKbzk5GCCcdwKhH7uCIAEfLkgY_z61Ij5GCeoIBI6GgB1q5F04JwBjGRzjFXZ2KwgnHHfHNZdo-bpwAcAAdfatCVQYgCuR0x2xigB7znfGuVwzYyTz0zxWdrhEXlkE52gEg4xxTbmWQavYqFzGWAJHqRj-WTTPE52CNcgZAPPIPAoA560lVZi2CXVjh-R6Dj6D3q7e3bKCY5C5B4wACQexwfX-tVFiURs6AEnABBzg9Tmob1VWUMpYBlDKD1PIyD7jHb696ANqyuQjFWkwhX5doyRz3_ADorMtL3zIy7oEQ9CACepooAx21CWJCAgVGO_O3rx6n05_GqVoxfVLeRzwJVPTknIqzNPcyr5TsPLjXy1BAHHfHrz361UtgF1W2GcKZVJyOnIoA-hoXzaq2OwFRyYJOcY9KISGt-CMAA8VG7bcsBnAoAxtQAFyoySMnHoOKyZUcIz-dgg8Dtit6coSwYAEngmsS7jABI5OeAelAHNzvJBd-erhGJJOD1wRjp0rUjRRapKcgMMlWJ4J579uaxbl2efaACxBBA_qO4raeNliigLEBQMkYO7Hrn8aAKy2SGTc4HlnG1gDkY5AH9a6GzgErjYjMccDGM-9Z6OGZIVPAOSB2_Lt2q_YzeVcKpY89R2x6f59KAPJfinLLe-JIrTcfLiAjUHpkkD-ddpPZDw74WhNlbKQpMQHphWJJx1JCk_U5rJ8c6E11fSzRj94p3KRzwec_gea67wtqemeItF-w6gSk8ZBkQEho3Bzkd8Hkg4xgkHjqAc54duLx9cTTbpo51lI3FgDgEEjBAHcDg9Rnpjm3ounf2B8TL3SLZglvfW4uRGR8odSAePcN-g9K6210HQtIuf7TN79oaMloxhAqnBGTtABIBIyemTjFYmiRzav4yu9cmUiMx-TBkYO3IyfXB4A9QM96ANHXtHBicAMDISSVOcEcmua0-xeO4mTBXC9xkkD-XfvXb65LNFFK0eCVBwDnGeAOPbn86wrI4uPML7iRg44z60ARynBQAfwgZJwP88VJuGAeMd8GmXQAkTAHHJx25NSgLtB4CkYxQBDagfbSoBwGAOexwK13AIQLxwAfrzWRagC-K9Bv6gcHArXcAAgLjB_HvQBXESvPCWAGGJHA5IBx_OsnxakjtGMcAAjnrwK2clWjxwCSSTnGMGsPxVcmO9RGzhkGMDuABQBjwGQsWwQpYqDg4PAB6e1WtTslMQlDEKcYTGeP8emahsHWchSWBUnaM-uOfb_8AVW1d7TZEAZAJGc9T3z-tAHOW8RMRUOAQRnBI7fSipLC4WElQOSMsQepBI_xooAp3SIl0zQ7SgO4HHGM8fhxWed0uqI8vJMq8gY5JHarD7y5VlK7TyCc5IJpQmbqAngh1JGehzkZ9-KAPcICFtyMcEDBprbQ5Gc8cg0tuS1vk4JAH8qSYguMYzigDKvEY3AAUYDZ_Ssm9z5DErjkjmtuUk78kbh6Vzc4mfzlZiCSSvpigDmzg3Dy7gpXgBh1raQYjjV1AZUHAJJzgf_X_ADrFtrSe51YW2ThuWJPQZ7fhmulMAWVizKrAZAI569PwoAdYvFIuWARyOFyCce_61oJBGzld-w5GDtz2HSoEhiYq0eARjIBHH19KeAxnywYAjG7jGKAE1G0hlljUMWcDGD9O5_pWFd6Tp5u8snkzjhXTIb8CMED8a6SQK8m_aSFAwR7cislw82thMjYIy-Mck4wDQBBb6PLPIoupZnRSCN7EgD6Enn_Oa6nTESIiJRsVMEYI5Oe4x7Vn2xkCCQ5Jxzk9B9Kv6W58-QNg4GARwefb6UAalzGs4KlQT1yRwR6D8656a0SCVkVBz0JGD_P2rpRgq4wxIGAD0z64rGuFMmCSFIGTnJJPegDnr_jZgAAg8-hyT_WmW77jtA6HIB607VHCRpgA4bBHsf8A9VVLVyZDweBnI_D_AOvQBPA_-luTgEuSQR9OtbGcwgtyTjPGaw7cj7XLnA-bufYGt7JKxkYycAemT1NAEUxYKhUjKg8Z7YJrmfFrYuoSSNwjUknr0HH6V10sQ2gHAG04xxxgiuH8dSCPU4wBgKqjB6YwOaAMW2d3vgq5G7gEHpXTXl25sU5JB5OT1PTp-HSuQhn2SKwIBwOhzn1rXmuDNp5UMRkAggAj_PNAFMTyJIRtzwc4A65oqJXyhLEE8DBbGOvHQ0UAWnX5s55zj1z35pYQXuYycZLA8jBzxnH5VMyBQWBDAHJA9cf4VWtnH2uFg27LjcAehyRj6UAe2wACyDg8lQMH6VBO-CCOoB6VNAVFkMnOAP5VDOSq7woCkdRQBkMSZmkZiF5BB6A1kXTgMSXHzEkYPJArauCRHIpiIJ5VwM596569EZkO4t8nTA65oAztHM8uqzrE4RihwzDOMEehrdjsJWcG5vWZhk_IgAPtznA9-tYmkoBrkxVTtCYJB6ZNdGAThiAB2Gc5oAcYGQHynAU9HJyBgdcAe2MUrl1iYAxM4GRgEfiTz2poQiAgMwUAggcCo8Mcgc5wMYyfQUAPheZwN8CKmc5WTJ-uCB-VV7idLfWYgTgSxMqnryCM8-wq0nmCQKAy57YwK5Dx2jNc6SsMzROWkBIYgtkAY46igDrUJLIsYYBhknOf0q7o5jieYzykMOVDdCCfp-lVLfMcaErkkc-_FSAruQsuTxg9fb_P1oA1LW5K3bxlNoK5Uk5PXOcAfX8qhvJR5hCgg55BGR-FMYkSRys2ABgFQASc9-vHNSXUYIEoOVYc4GDk96AOZ1VCQwweDkkcDt_jVS0jIYE_ebAyTWlqSErnB7jHSqdsBhQMg5x1oAbbD_TpRkj5hkD6CtxcFYgRwrZI9cHNZFuP9KlbGMsMAjnPTNa0IBYAEAY4A6mgCad9zjgcAjr2xXBePoib5SQfmjB_QV3km0FQc7iME_nXG-Oj5t8owAVjAx3HT_EUAcUHYeWCDhVxkdK0hLm32oVIwCCT1rNMXzkkE7R0PQ-1Wo0cxAAZK98dOPT04oAmjiVnYM2G7lcjP5fU0U6CNmkK-aVOMkD5fSigCxM0sSBIvmLMcnpxgA5pbNSJIQUBHmAcd-eue_Jz-FWJY0L_ALzcGBJBXg4xjp3qIPCtxCCGG2QZA4HA_nmgD2uEAxbTzwP5VBeHYqKoBGe_Sp4zmJMAAlRx-FQ3MeWG7nHIAoAxWjeaVpDLtHKlc8dK5e9IgecB1YHqTzmuqzGu87dxBOQfpXLXEKSpO7AqQx474oAbozql5J8wIMeCcdDng1rM4RgY5CW6H0ye4rnNJB-3zAE4CEAE9eRWnI4QIoJJBz6_h9KANe0lRVCkkt0JxgGrGGLMMkDIIJOTWXASdhOBk5I61sI6GY_KR8o9hmgBlpKsVwzOGYkEcnt-Ncr4wSWbVNNmRFJjdjjgDAIODXUbQ84AAIBxisXxLKkUungKGaS7EPPoRg_XoKAN0wKIo0YcgAjHXpSOpxt3M3OASelXXjTgOSSBweuR9KhbETEsQT1475oACSIAwwcgEjPJFWY2D2xXqRyP8KqB9y5yCccKeePpmpreQGMrgLuGcY6Y7CgDJvo2JJY8AZwf6VSt0YKBzwSCR2rSvFUsrMpBAIHJz6-tUIgY3Y88HIyOvagBFYC5myQQGA4-mauQNsuY-Tg8ZxWci_vbgHgkg57cir6DLBsc8HpjGf8A9VAF-Rlfk8kZI_OuI8Zyg6gM8BlAPqOB_gK69nwSSTkdOn-e1cR4vJfVgoOCoHGOvGKAMD5CgUgZGSW5PpWlaRo0bBhnnAI6Gs_DNgE5ABBA4AHFa1ghMXABGAAfx45oAYYRnc77AMqOOvSitKWzZyGABJ5JooAydQlMd0CgySODjgA9s_nzTYVaQRSkciTIB6gcf1FToFeaQSncCOCQO2QP6frTo5RFIUCbiWAUk9OefxxQB7NAR9nRhwSgP44qNpQ5IxyOuaLUjyExk5QZJPtTZAr5U8E9cdaAMe4TEsjEggkYGOlc5dwStJMArNkZ4HQCuouUwrFmHtjvXO3LFJHbeQdpGAcD3FAGNpoQao42ksUIAz06VfYg8EqR39RWbpjH-3CCDjBAJ6cVpMMTMAc5OR1NAFqE4UAckDJ46-hrVspiYyhySQec81jRsRIrrkZAGen6VesmxICp55yT_OgDSjQPONwIUn5iQcjPcetYHjGREXSdsQZv7Rj5JwQCDzx6YHFdBnC5bI78d_wrnfGBfzNKRl-YXiHB9CD_AI0AdKjkRAMA2CRknJHNVp3KPhjuRgcEnOMdv0qxLF5WCDjjkjgZ71RdGAcL82CGXJ69jigCxEVOC3I6A44PH_66toFRCQcEcHPU1Tt8sRwQR68DHcGryKBEVPIxx6DtxQBl3ODGGIOAc5B_D-tVgOCSMEEE49P_ANdWpwDGQRjjn8OapuxABX7p7DpntQBHGm6-uABwApIPuBVxwEZgTgYAB_z06Vn6bIZNUuxnjC9T7f8A1qtzk7mBGCAPw6_40AWR84U5wD69-lcV40eNdVDJnBUZ-oArsUIKovOAuQSe3AriPGcmdUG5eABwPp_n86AMmOMtliGK8ngdPrWxpyM0YBO07gMcDjk4_X_OKw4JQIgNwBbIXJP-eK6HSc7kYuGAAGR0P50AbccYX5im4Hsxxg0VoW0e9dwxzzyfpRQByJhBt8gkkN1z04659v61TkRjeQoo4JyAByTyR_OtOLatoyBcsVBbkcYxge_X9KiEZgmt3GDtAc4HI6nj_PagD1WzBW2jDdQg_lTLhgGB3YIPNELs1rGxySYwSR34qKcEorEjk80AZN5cogJVizE845ArDAe4lkDKDgHknqTW3JGyysAi7MHIA5rI2souZApXaSc-g7frQBjWEZTVlRlO4FiDjqO_1rWnQlt2cEHr1rMsZ2l1mAjaWUMCN2DjFa1ywMhCqQcYxn2oAZEWwCSenPH9K2baJY4POdtxI6AdDWKSI488nJ6Z_CtGzuGnjCMc4GQQOvYZoA1YmV2BIJJBGMVzXihGN7pxUqQLkMQRk9COD6D-tdJDyuQSADkA9qxdfhL6vYKW5UEgcDOc9vagDoZAskRZlySAeQB9c1mzqABg4PYir28tEA2CRnPNVLrBgZtqgggnHUD1oAIHJUAkFhxx1I9frV23dslWyc5OSOlZyASmM8IwGeD1561owsdo3DJ6AgYNAFKcEF-gIzx7YrOBYRDCjIGRz6Vr3SBmkzj8Py_pWaqBFk5zgnnrnnrQBm6a5TXLsE9cdenQVozEgyMDgEA5-hHb6GsaFwmuXDA4BwAcHqFFakpxbswIIPHPXtk0ALbu_nxoSBlSST0xkcY_GuR8cFjq7ZIAUKMYwDkZrrrIJJcRluXVcjmuS8bqTrMuQRhEOc5zwBmgDAiVS6bipAY8Z5HGa6bSAGwAASCDjBPArmrcB2Vs4IPI9hjNdZpEStIhj3AAAMCfb_8AUMfWgDq7ZC0IG5RjHXiim2cohJy-PlA5OP8APSigDkZFMfMYP3CrAdzzz-ff2FSQJC88bOFCAqm0ZzjIyT-v50y3AMSkZACEHjp0Ax-RocBGSQAEA44OckduOnSgD1LYEKquAoXAFMkRgoJAx24pLN_Ns42Ay5QHk-1WhIRGA4BGOBigDBeNjcyYO3PHWuduWl-13ETMSCcnPANddOIDcFkBB7gniuS1AxDUXwWJI5A7HtQBiaaBH4ijCkHBYEEYHT9a6GdC0mQADjPWub04s_iVDllAZie-cCuokwXBPBJxwaAKDucELgBegx0_xrR05HMRcgBCCCc8_wCf8KoTc8KxOTkZFaOnkhNgAwBycHNAGvakhlU5AOMg85rJ1nP_AAkVjhTtAILE8DOcAe5wfyrat0AXKgk46kflWB4hZ01zSo0Bbe7HOem0d_wJoA2GJVAABg8jFRYbaXJ5x1Izn8KlcNgNjAwRnqPWow-FAPcccc-tAFcCKNwzkD_ZI4z7f5xV2MkxkRt3yAc8Zqu6biCuNwPGen-etLkxgsOOMjA7elAFmUZU46lckjpmsmciK4PAIPJxxkf5NalvKJY1KgYwRjr3_wD11m3ykOc8HHB6ZoA5l5MaxMpIBDEc8EgYH8hWmtyDYjdjJPp_n0rFu2KavIRk85HAOeAP8amWUG1KAgMoBweuCcn_AD70Aa2lSGSfJPCqeMc54zWF4yKNqcyk4HlKASehwa0tFlczOq5OUbAP1_xH61i-L2YavKgA-ZQRkY59c0AYFlKMgE5IBwQOT_nFdZo04VkGWwBk5xjJ_wD1CuOgRVjVsZCk5Oeo_wA5rf0kSKwdWG3oAQCD6YoA7SMlnATagC-nvRT9LhMkQJ2jg5785ooA5-MRSQF1XaMBeDx0yT9elVgpVgRkFSMn1yRRRQB6fpYzp8JOT-7Bz36U922IpJIzxRRQBm3ETPcBgxAPp1rn71YY744Rmkwdx7Z7GiigDA0_MevxsCDvLblHGODW3dTlGChhgDJBHXOKKKAIC5bZjjGOM9a1dMYZZTjJIyOgxRRQB0cO3aBtyAQcg859q53xLIqapo8-duZHUYzznA_XI_OiigDTYBosYPQdO1VnQh1-Y8DnB7UUUAPTJbBBOPTvU7LuRjjIAOOcGiigBli-xguCMEg5A5qrqkREm4kAEgAHnPrRRQBxGpuPtbsMZDkEk_jgn8aapYBSqkgxDg5IABzx-RH50UUAbOgBmm3YGAh-YHrzWX4xjaTWX2jBWJWGe4AB_wDrUUUAc_bxJuIJKgA844Pt7cGui0y0CSAFF2smAMHGcg5FFFAHV2rFYwoC9MjKnpRRRQB__9k=
https://data.labs.pdok.nl/rce/id/image/20324688 http://dbpedia.org/ontology/thumbnail _9j_4AAQSkZJRgABAQAAAQABAAD_2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL_2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL_wAARCADIAIIDASIAAhEBAxEB_8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL_8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4-Tl5ufo6erx8vP09fb3-Pn6_8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL_8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3-Pn6_9oADAMBAAIRAxEAPwD3FEwAD1pWjXHQVI4A5HWoyw6UAIqD0GRUyog5CgH1xUasM0_eMUAI6K4wyg_Wo1tYgPu5J9TT9-TTwc9KAK7QoGIAwDxTwFUBQOKkwST6U3ABoAjliVkwOD2qt5DknI4q8Oa5_wAT-IY9FtCiENdyDCJ6D1NAGL4u8QjTYGs7Vh9rcYJBzsB7_Wue8Eau1pfvYTufLuDuQt2f_wCuP1ArDmeS5neWZmeRzliTyc96fcSmWWGVVCvGoGV4JIyc_wAvyoA9cxTWqloOprq2lxzEgzKAsgHqO_49a0CtAEBFAXkE-tSFRnmpIkBOT0FADfLPoaKtbB6UUAX24UkmoAfmqVs4NNGDyRzQAxiAeKaXOetDkbqYTQAomHmlOdwUHOOMHI_pU6ydKy0nRtUnhAG-OKNmPf5i3f8A4DVwE0AWi9NDA1EDWNq3iKDSr6O3lzl03AgZA5xyaALmta1Do1kZXIaVuI0zyx_wryu-up768a5uD5kjnJOeB7D0q3qt_Lq2pPNI4ZQSEA6AVTRCJCGOADjpxQARPaxSFrmFnQA5VDg59RyOlS3UWlyRb7OabzMBvKlXsevPTjND20bxusrkIc4IGSPwqJ7aJFBjuUdVXBUghhn2P0oA0PDOo_2XqYVyRBOQrDPQ9j7V6QVyMgjB6Yrx58odygk9RXovhXVv7T0sRuf9IgG1gT1GODQBrkYpyvjtSsOTj1phGDQBLv8AeiouaKANQuCKj3EA4qEOxPFLlsZwcUAOYkmmcnik3c0uc96AOe06Yv411hQcqYYlA_3Qc_qxroxXH6BL5njDUW4xIJCPoGUD9K6q7uorG2aeZgqqMnPf2oAqa9qjaRpE11EqPOoHlRscBjkZHUdj_L1rzZr-bWZZL67eOOVl3LEWwxGOcDvUPiTXbrWdQUIMxB-ASCFUEEk-5yQPrmn2EELxQy3N15aALHwMlVCjnH-etADbdITDMZCQ-P3Z56_hUYDYwGIJA61PcW0UUzC2uVnjGMOBjI-lNiVsgEkL1JI7UASqG8vDEHtwKR4cZLKcEdaeXJbZEQoPcjJ_-tUU0k4IaZ_MQALuAwVwMD-VAEThcH0HtVrRNSbStWjmBJjYhXUdwf69D-FVZyQBggg9D61FII1t0K5EwYk8nHsaAPXg6SruRgwPORSEc1zfgiWafTrgzPI4DgrvJIAI7A9PwrpiDQA3FFPxRQAJOiziIn5ypYDHYED-oqwTkVyXizVrrRjBPabN5YKwdcgggk_qBWRB491EACW0t3HfGQf5mgD0Bl5z60ydjHbSuOCqE5-gJrkofHyEATaey-6Sg_oR_Wp5_G-nyWUyrDOJWQhVcDaSRjkjNAHN-FL9YvEcEjzjbIsvmEg9ME55HqB-VR-LPEc-q3H2W0BWEZwewHcn-grCjgFsW8rYbplAJycRqARk_mc9M_TObCWCz6cEtXkllZjuCqckg4P54NAEAt4YtOR4ZtzswJIPJ-Yd_ersSg2kZIOc9cegAqeLSEiso4bmZY9oAKINzcHp1wPz_CnSLFH5aQoyRjgZbJPueAM_QCgCvEuCxI4B6A9atZJUKOQeT_hUcIGCQAQDzxVhMCTAAwBnp7_zoAswQQOphLDfIvBzgA9h9c1XvLKeBSJ4yMA4AI5x1P0qSQw7WJA8zggY55681HJK11eqbp2CGPCkHABA6d-o_lQBQaIrHsyMHleenqKi2B5ER3CAsAXPQcjk1cn25jZCSucYPWqt2MxkgDBBHFAHpHhW3SDRUVZUlO4guhyCB0rZIwa5zwEAdClAGMTnjrjgV05Q0AR4op-2igDhPHLrJKsYIyrIcZ68NzXLrEDgj0rfvydTu577a6RqmcZAIUE8EEHnn8KxpLhIzkmEKThTIuCT2GaAGpbtKxWMAkEAD1JIGP1qaTS7yAgtAV5wC5AA9STSRkySoAEUlgFMRyWOQQAPTuT2HuQDYnS5KmBy4JwSHyDx7HmgCCOGytoJYipuJJANzZwM4OeepHOKkS4cRiKMLFHyAiDAI9-5_GqrQujEOxHepUQlgQSQKAJEQliSc47USg7o-gAJwKsJEAec5NR3AUGMA5O7GOlAC2ZwrEAHJPapG2-btUAEg4z9aZageUSOMseD9acQxugQMkqTx9aAElRngBXBkGcgHAxUAiLMglB2EjkAcHtmrCA5YFSMdhTST5ZOTgd8UAQXCAuEGQAfTt7VXuogISQQQKtS8mPJyeuc1FdAeQ5yOBQBueFdeXSAba5X_R5WDBx1UkAcjuOK9BjkjniWSJwyMMgg5BryLZmBDgnAGDWpouvXOkTbTuktiRujJ6e49DQB6ViistPFOklFPnNyPSigDymWdbmeTyw6BmBwGOADgkficn8aW6gD28YLkBW3YzyfYZ_nVK0LYG_gkAA9zg8Z-oIrWsoIk3TXBLuHICZyR6D2GcUAO0OwL3k0-zywWByuRjAB4PX0-pPNdNeWkA05btLtmugACPMyRzzg9ax4ZQ8QUgALkADjk5Ofrzj8BVOaV8RqowC2Rxzg0AaAub4glb6YqTxuCtx-INU77X20-VIp5Ecv3Nuhx6E8D9KtqcADPt7Vxnid2bX4QArAKvXHHNAHaLczlQfJs2BAPEJGR-DUzZJe3cNskEMchJwVY4PtyeKW3lP2ePGMbQP0qG4mZZ4njYqwYEEDkHNAE72ktjK0EwG8HJAORzz1qFiROpGeFIqxHcyzM7zyM7gkAtyeOlTJA9xMXjhdwFwSoyAaAKSSMC5GQDURci3Knvk1p_Y5gz7reQD12Gq5tmEDFoXBGeqmgClKcmIAdgDx7VFdoDA_GMDNW7hEiSNnBUDGSeAOKpX2oWsUDGOVHccgD5gSOe30oAtQQSvaoVQsAB0pkltcA5ETj8OlQ3uvaZcukqwvA4UBgo4JHcdKz5NegXlFuDj_AGiP60Aaeyb-63_fJorK_wCElH924_7-_wD1qKALUTxNexrtdcnchxgEYXBxj2PFW7VTJLMSSSZSSSeetZ1mf9NgYEABccdhhsfyq_Yjd5jYyTKRn8TQBYUqpKhg2CMbSCDwOQR1HHX3ps5AEBAxk8nHtT8AM6joHxj04qObpbjoQeKALSgEDBxnnr_9aqF_4XsdTvY72XU2hkVQCn2csOD6jrV5QOCMk1IBnqKADykhUIkgZFGA54BHrg1XlAaeJQRjI5z71l-KgRpqEMwO44wcdqzfCe5YZCzMx80HJJOOlAHSX88tjp9zcQpE8ykBBICVySOwIJ4z3qPw9qd5qcE5nWGJ1YbTBuXjnrknOeKh1tydLuF3hQXHJ6DkVW8Ms0UE4DhuRkg8d_6UAdSlyImKSXVwXUZYCQjA9etC6qvmrEtxc7nXcoM3UHoep9DXnfi-9uIL-MwyuhZQGw2Mj0NVVv4gHLGTdAoBOTnBOKAPQtW1kPpc5MskkZPlkFwec46Y_rXGtt-YADB6c4rmo7yU6oI1kbyXkyUPQ5710BcYOKAEYgKQDk-hHFViAepPPapGPOM_Wm5HX0oAbsHoKKdn_OKKANRbu7tLnIVJArZZSM8EnH8zW5pU9lPCC0jW7mTJyMqSSeB3FVobOe7BmW2IflSOhOMjn_Peon0S_WRJoIWR85IB4JoA35dNuI2kcJ5gMhAMZDAcDGcHIz71XktLl2hAtpjgnkRmq1omt2kUiokis7hmIYjP5Vau7zVnAjEtwgJAJLkdj05oAuJYXeM_Zpsf7hrkPEniWfRdQNsQE46OMEdPX61ufbdTRAGnmyc4XzjzjqaydT0q31W4W4vLOO4uCoyzuenvyKAOZu_Fg1KPyblwFJ42gZB9qbZa-mmLIiOMltwDDnHarOu6FY2ot5La0jgKgk7CTnPTqTTtF0aC9jE01rDN8wUmTqBj0_GgAbxKdVtJ7ZwpBAbI4III_wAa2_DJBtZSvcjnpk85_lVu10iHTj5lpY6coZQH3oCDg55BFXC8s2UtxaRqMcLGFye54HI5oA4vxmMalbjGQQKrGJGa7KgjcF_ABq6jWvCjamy3L30SOmAABkE1S1Cym0u0aUSQozZAZSGJODwAfYGgDjI3U6spA6ODz6YrolkycHtXIWyTTXjTyMW2tkkd-a6K3kZzkUAX-uDjikJwOhxTcsCOh9qUnjBzQA7cPX9KKblaKAO08L6sl5JNG42nIbGeeQB_MH866XCEYU5P1rza1uzaTRyQoqELgHkjtV2XxLqRbAuVUdchQf6UAd0yEkYOMg9Cev8AjWHq8WpowktWDquSQV56Vzra7qL4BvXyQORjAz7iom1e-wVa7kbkcgmgCGXUtSTAZ1V4ycAjoD2zVX7bqLoDvYk-g7Z6VbkuZXJYyMxOSSTn_PSkMvUKWPXJzigCrdtdT6ezXBYkHCk-lS6XpuoXdorWYcruOSGwAfrT7gySadMXlLKpJUHGFyBUNtaPfeXAuvPp6FQBCqg7jzk59f8ACgBmopq-nEi6L7CQAQ-QfyNZj6lcnK-c3UYOSMV1UVloWkwFLzXmnGcv5jA7j7gg1m3eqeBwSkazTSN0MOcg-2TigDEN3cOcNMxGPU0rTzOoDElSOh5qzNpccy-bYOzKRlUZgXH1Axn8KoeTdgMpjfC8HKkAfmKAAJEDnaAc9hjFIysDlWIx0xxTRIBkEnOfwp5KnBDcfWgB_wBquo8fMGwO4zT01GUYMkSse2DjFMY4YHaME_pTvlIJ6Ee1AE_9pr_zwP50VW2j1ooA2WJyGBwo4Hv-FRliELA4JPA9fwpuGUgoCCQCCACSP_retMZyCEAZieSO_vQBIJcggnAJOAO5wO341ds9Lvr6LfDGCnQknBrPwowQoBxznrxzz-dRRavfaVcma3nJUkgo4OGH07UAdInhvUT_AHE4xndSy-Hr6BQ4US8HIQ5Irlbnx5rIkKhmJIBOFOASAcDGOnSnaJ4l1nVNat4Z3mELP83UZA59aAN6ezuINLKso3MckdCB_jVP7LE8CSsAcHHB5B7c9utdD4gu4_KSFWUZALEn8hXMpcxI0kRlXaQTnPGf_rGgCCXTLCSRpHtQzdckZP61raXB4eEgikiWOboAQADj3xVFXdjgAs-B90Zz78VIdKmuUH-iSkg5BKnGPb079-9AG9LqnhzTfvPbKV7AAkfnWXefErSIQyQo056EAYFYU2gWxnP2m0IcHkSHt-PrUlrpOmiZVMSQ57sMj60ASSalaayvmNaRWhY5VkXOc-oH-FVJdGuFRnhAmB6GPnH4da6-28OWIVZC5kGMApgA_lWpbWVpbEGGBVI4DYyfzoA8ya3uo4g_lSFBnJZDgUxLklTlScDHB6V6vLLFEhMzRIncuQB-tctq8vhicmMgNMxwPs4wSf5GgDlvPH-TRVz-yIn-ZZE2nkZnjzj86KALDSsFZcgFfUdcd_T0qe202-njRxbOVkyc44x685qh5-7AALMDk4xz_hXY-Gb8XenrE4AkhJU_TjFAGOnhy_clmVQc5ALgAfgPqelTL4UncgvLApHbk5966onBIIwDQTkYNAHB6po39nygyhWBBKsBgEcf1q14aEUmvRAKuWJJ6-nT-tdRfW6Xdq0UyBlJGPUe4rndLK6HqxSdFHmMTFNj3ORQBF4js2GvzrvHl7VGO54rKgtogC7IGYHnJznFbmuzGfWZZdhwVXHIAyAB1JrKkkjiLM8kaqThhuyceowKAN3TvFdhcIkTMInHBBGPzH-FbSyiRQQxIPQg5FeTX9qnmu0Eu8KchhwRnpxWjoniC9sJkgnkEkfA3g8D2PvzQB6VJDDcx7JolkX0YZx9KxNR8LQ3KH7JII2AOEc8E5z1HNc_c_EEJuWJU4JAwCScfU1lzePb2WMGN2XJIwAByAP8aANQprWhOGUsY-CQSSpHsaq3_ifxFcsVt0WFT0xwT-XP5EVTi1me7UtLLI2AMgsSBVwSjaoUsRnGe2KAMKew1i7k3Xt84J9M5_U5qay0IW0nmmV3YnIJ9vp_nmt3JkUMSAAcMOo59KFAI3qxCk4GBx7nmgCn9k_2n_76P-FFWPtco43t-YooAikOxiyFQAcABeQB0z2yef8AJrQ8NX7Wmrlp2YJKh3E5I45zz7f4VUWeMRFCgZyQSNxHBHTHtWXKzITIoIYkkAZ9wKAPQZvFenLIQFkcAdVXGTn3xWbceO7OEnbCM9i8oU_kM157qBuJ5iFRgMKMcjOBjOPfGapf2VNKSz5GfbpQB3U_xBdyPKEIAP8ACpY_mSBWfPr9zqseDKSAd2NoXGT7ZrnItEmwmX4JwR71p2tmEDDcVx3J9-aALJLOS5ZndjnJPT-XXHpTzbkwlmAAOASSOOBx04_CrdvEpj-4pIIGM8nn_wCvUgjEjuu5SC2cdR0HrQBUSBZFjBViDzkDOfoe9R3Fovm8FVLKeoIBIHsME5B4AFXreBwdyHDR5XAOMjp_I0k6ANuUgkE5BPQd_wCdAGLJosAIf7xORtBIIx1GMU6DTVUiN4ioUkAquckjPOfpW-0Kzq4TLELkBAc5_Ciz0i7mYSLAFUvglztAIAzkn3zQBmQ2QQgsAoIyo6dO5q6AsSghxnHBwCCRnoe3_wBarcumWcU6G61K3jBTdiP5yORwPfmq9zdaRFGEt47q4lJ4LnCkZ6juD_KgBsaMY2TcGLAEk4BB6ke31GKuWelXckKPDauyAljlQFx1HzGov-EjngtyltaWtvhcfKm5umM5NZ1zqF5cqFnvJHwudhYhT2A44oA3ovD0vlJzb_dH_LYf40VymU_54v8An_8AXooAvqJTGQIYuCf4skA9xj0xj8KYEjSUFQrcYJOQPzP-BoooAJgouDgKobkjGVBz9OB3-tJsQxbAAzdz2B9MCiigBOpAIDKepORg446_54pYEXzHiYAAHJPcA9MD09_cUUUAaEdlJLEDFHJKSdo8pSeOxz0B-pqz_Yl6DMXEUaoASZnC8kHgY9gKKKAK9u-lW1w_2rUVKjokCknpjg8jt60y51fSYgVtNMuJyWDb5mxu9RgZoooAiHi3UVUxwrb2iAdI1BYAdOTk5rJuNTnnLuZ5JgWLFSxGSc5P1NFFAEUUpkAYxmMkZAB4x-FWnlbAYMAMYwTySaKKAIYovODBmcYySTgn271NJaKxwsoIGCAXzg_QgUUUAR7X_wCfmP8AMUUUUAf_2Q==
https://data.labs.pdok.nl/rce/id/image/20636163 http://dbpedia.org/ontology/thumbnail _9j_4AAQSkZJRgABAQAAAQABAAD_2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL_2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL_wAARCACFAMgDASIAAhEBAxEB_8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL_8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4-Tl5ufo6erx8vP09fb3-Pn6_8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL_8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3-Pn6_9oADAMBAAIRAxEAPwD1LXtJiS2-0pMIZywH-zk9AT_Wsmx8U3ViJLG72pPkFZZclc_7XsfUVxE_xCuNU0trK-G-QkbpVYgSBTkYHABz3796uX-ox6hp1hcxRkER4k5yUPZSfXr-dD0sNam9NIYbyXUruaJvlCxiLAGB2GOvOfzrmDGHkaRsEkk_SqQlLSccAVaEoVCxOABkmjd3C1h0pAQADLscKPU0kNv5KbSdxJyzepot1LMZnGGIwoP8K_4mrsaGRgoBNAFdisaF26Ac1XW4dnVxlFHbNa2t6DfWCQvcJiNgORyFb-6fQ1jbccDtUsD0yLU5NT0K3iEIeO5XypJFYDyyCN2QffkYqjoU872jWvntG9pNtli4IcZyOvTkEcVieGtVFqJbSVsI7Bkz2boR-I_lT59S-weIpZ0HyTBQ49Scc_p_OkBpWKw2-v39lIild3nwEj7ufT8D-lY_ioBta84f8tUBP1HB_kKh1TUGOqpcwSDKqB8v6iqd_ere3CyBSpA2896B2C0iE95DF2dwD9M810_iCZxDJIk8m24ZUEIPynA649cAD8a5a2lEVwkmSNp61fmvzdahb5-aOJgV9-mf5UnuFjqHv7jSNOltkiXyYoABKW_jKgEY9cng_WodCg0z-z2iv1RriQCUKT8wTBwR37ZNZOqX4uzDZBsBpA0h_l_M1e1a9SGwZIVHnTgQpjqF4yPyAFAalfTNFfWftMsEqxRI-2Pfzu__AFDk1kXsRsrxraV13p1weK6G3WPSrcztNJ5UMIDQhsLJJzgn8Sfwx6Vx9xI9xM8sjbnZizE9yeTQgLJPHWhR371SjLRFsAkHtV8KwVGZGXcAwyMZFMLHP6pZfZpfNjH7pz_3yfSqIkdSMOfpmurmjSaJo5BlGGDXK3Vs9pcNE_I6qcdR60xj1uZB1IPrkVPFOZGCFcE-lUlPvVqyGZ846A0AXwMCipQmeaKQHnUd432eSIyrtByEI579DXQW81za6faXdqA5EY3qDnIyeGHf2NcjFL-7lUSAZOdmOT15zWotxJClu8Mm1xbp368HqKqSvoSnbU723mt7yKG5UFlmAfg_MfUfnxn2qPbvmGT8iHJX1Nc5pmrzPeR2yRgecEO1CQCzHsOxJ9Dj1rfd4-VAZJVJ3K4Kt07jtg5ya5W5035G6UZI0o2LYHrXUaZpGowWserW8asyOCkbDJb6Dv8A5xWJ4a01tbvDbhxC4QuCw4I9K6U6ze2cQ0jUpRbrkLHcqnGFbpnt0xnt3FdMZJq6MWrGhb6wNdnlDxw-QYfLlt3OWJyefoP84rj9d0kabdZibdBITsz95fY-v1ro9b0_zW_tG0fyr1Bu3x9JfwHc-veucvNbmvbRop41Lk9QOAPp60CRkBijbgcEcinPLJMxaRizepppHzZOaBSKA-uaTvTsZFJ26UAO_GnAkU0dadQA5ZSrh-rDnmpTcM9ykrtjbjHtiq54ooAu31-9wgiDfuwc_U4qkBk0nO7mrtg0SXCySNtCcijYDV0_SRaWhv72BpXGPKt1GSefvMPQdcdT-la_iS8srxIbWCNnvEOMqMbB6Edz_Ks0apd6gwttOUx5-_M3BUf0_nWrpUEWiXJkMO6FI9z3TkD5s-h6DHQDJ657UhXsck8bRsVcFWBwQRgiql_ZrewbeBIvKH39K6HV5Bq13PeWVpIsKAGR8Egn1PpWP04NMZyG0pIVYEMDgj0NX9OQmRz7AVb1Wy3j7TGPmH3wO49abpKZjckdWxQBdCYHSipioVSxIAAzk0UCPF4n_cSL5nf7m32POf6e9bQ8qSG2R4wSYUw6HDA7fyNYMD_6PIDI3f5NvHQ85_z1rajliSG3LpJvWFTlSCD8vGQen505CEtDINYjQJJNho1CR_fIA3Y_nXSJeLM5WN2lAPMU5xKn41yYmMGpGUsyhWU7lYgj93jORyK0vtXnqGZxOM5_e8kfSReR-NCV9RtnY6VrdxptyJbSRg6MGZG4f_6_Wunt_Fmgzae9vqBuWkmdpJFaNpCGP8QYDg_lXmC3bKgDMSv8InP_AKDIOPzp9xNHNEfOUlgpG2UgFh3AbofwOanlt8I733O9s9cCtImnXck9nFIU2yIVwcZxgjIqvJI0sjO5yzHJNcJHqB0m7BgJKEbtrfxgdQ3qcdG68YOa7WN1kjSRCSjqHU46g9KE7q4WtoSZowfWgc8UZxTAPpRSZpryRxrukcIOmTnr2HFAEg6U7GajZ1jRndtqryWPanBwRwfccUDHY5o-lIG9BThjHWgACgDNHQ4oyTzSHnmgDVsdTjsIfkiJcn5ucfrWnb21zq7LcX8mIRzHbocA_X0_n9K5mN9sikqCB2Per7a1dBWjhPlg8Ar94D60ibHVjV49D0v7HeLA8hDfuYznOScEjGF49Sfxri2lWVyyrtUk4X09qgYMzEsTk8n1NSQD5iKYyYAsvQVWtYUiaZFGAsh_kKugcdKghGJ7j_rp_wCyigDM8S3RtNFl2th5iIgfr1_TNFZHjOcvdWlqCflUufqTgfoDRUvcpbHnMT5tnXe_BJ2bfl-ufWtOZkEcY6P5KYJ7fIO9ZUD5tHG9zjPy7fl-ufWt1ZpQkMJkJj8lRtIBGPLz36VbdmZvYm0eQR66CAW2QpuC4J-6fXj860Ne0uFLZLuFFhmWQqfKj2FgecfUD-dZVgAdfkB2YadlYPnaR5Z4OOa27-8gOmT7HVkjAbCswwAwCnaevBOSK55NqomvI1STiznZZbq1mYJJ5sZAZXxy6kcH_wDXmmm_Cff4XG4xAfKfQkdPyAq2lndSJvSDMZz8qkHrzkDPQ_l9DWfe6XK5UFJIyDkFkPy-v1Fb7qzM9BTPLqd8iRLl3-VF9Se9enQR-TbwxAcRxqmc9SBivP8ASb5NDWUiO3nlY_60gggYHFdVoevRaqHV_LjmDfLGCcsuOvNGmyA289qXOenQCoycDjNVtQuY7TTLmeX7ixHIzjPHT8aCij4j18aRbCOAq105BEZ5AUHknuB_OuK1PV9R1GSGK9lliil2uFCFUCHo2BywrNVzdNP5807zttKYG4uSeQxPPTp74rrtJ0a3s2tLq7aO5sZ8RONxKwSfwhvX0OehPSqtYnc5cRyTTGGa5KwjdtkIdg-MlQF6jPH0zWpYa_rMFs4geSWGPCt5qbhHngAN19cD2rdFhYN_ZkdzDCsVmLh7klAMiM7Bu7nkg1i63pMNhFFdhzDJcsXWyJLMi9VJ78d89CeOlG4zutM1CLUrKOZJC7YCyBsbg3oavqe1eV2OpRwa5byuZPJXYJw-Bk_xdOoB5H0r1Mc47j1FIY_oOlGMmlAp1IBm0Egmpo4WkYKoyTTAMGrVrM8UihE3-3rQBbg0G7mG4tBGvUl5OlS3ugTaZax3LyJIkpwpRWA6Z7jmpjHd3sYX-ynYKQwO_uDkHpyKua1e6jdWcIvLWKKMNlChyWOPqaEIwAOKrxDE9x_vj_0EVaFQRD_SZx6uP_QRQI4LxLNu1-5kOMQjA5_ur_jRVDXZd76pLjO53HT_AGsUVFrmmxycB_0RwHkwOSpHydu_rW4MGSL2hU_-QqxIGxYuoeTGOhHydR0PrWzGQXQE9IP_AGlWkjIfBhdbmwRlblyAR1IQ8VaIxHeo8-5TA2Fbt90g8D8PSq9sGfxFKikgm5bkf7lXLyxng06-YOpLR7RhWwcFQTnAA6fzrCT1t6Gq2IdFuDstl86zjzC6lpl6KGGAeepzwcZ61sLdsnS4sOfSVlrFs7q0tYVTcWKp5YHlA9DksPc-_b0qdtZgjwRE57D5VXP6VsjO5cuZopELtJZ7x3FwOfrleRWBcSCCaG6tiI2I3jYcgMO4PpVu71WQ5QRMo7gOMn24FR6Xplxrt0rlfLtUO1m7Af3R74pbu6Dod_DKZoYpCNpdA2PTIzWD40hEmlwuXkG2ZUwG4IOe31rocAbVAwAABj0rI8T6dJqOkP5buGhHmiMdHIz198ZqijjfD8clzqlutqbWCWEkmSQFhJtywLc9sAcY6CtrxDqMltZysGsvtM37uV7SYsHHfehGPoc5Fc9ps0wnikaEXdtZAERuFxtZvu475JPrziuunt31jTLdJvKQ3Zxa2lvwkfq7kfeKjtwAcDrTYjhIdQu4ZhcLOzuWDHc27JBzyD15wea7-wnjlRwl1pkE064me4laWdsjndnb69OlYVp4VRp42SQTK5mMKMMCQxsBtPoGGenStm_voIdE8_EN7ayKyRw3QBmgcD7uT94KeoPI9TQwOGuHEZ8jZEzbw_nj75GMAZz07_WvVdCjEWi2YUsd0KMSWJySB615oILmQpp-0BZZEkjiyGwWAAOR3xj9a9R0-zFhZRWoleRY1CgueeBjj2pMC-KXHrTRTu9IYDrVy0uPJkA8vf8ATrVIHmr1ncpAw3qSM9aGI05LmG4jVHtrtcMG-VRg4OcH2q7rGr_2lawxfZ5IvLbPzE-mMAdqrPqVtPB5cd_NbvkchG7HpwfwrQ1bVbK70aKCI_vVkBICYGMHocA96LCOc4FQRD_Sp_8AeX_0EVOOtV4uLuf_AHl_9BFAHnOt2cYs7-UO_wB8nHb79FQa00uzUoyxyrtxn0ailco5OM_6ERmX6EfJ_D0PrWxcXUyx-WZpfLWJRt3fLjaOKyVBFkcmTGehHy9unvWxLbkqWMsI-QEJncx-Udu3TvVO3Uh3sWNLiM2r3ahWeR4wy7W2ney9j2rZ1hiumEgbg8ijmQsTjoPTOePauZtrm5ttVma2ZPMLlAXGQFC__Xq5c29zcGOTUbs5GduQE6-mRuP4LWPs25qRpzWVihMyRSsGILISp29M98ew6e9U7qOa5I2htuMk_wB729hXQR2FtEoYwkHs052BvoDlj-AFPmtTFEbgqke3ndINit_wHlm_HFa6ogzdBvYLGf7Pf2kLws3zM65aPtn3HqK9Ftra3tlK28SRqxyQgwCfWvONTVWuV2JjKEke3-cV6LYq8dlbpJw6xqG-oAzSTurhazsWiARimkFgVORkYpw5FKR3FMo838QeGDpSm5iLSWxP-sP3o29D7e9V9P1W406eKS3aSNpYxG7SkFSxPJB7DOD1zxXpc0EVzC0U0YdDjKsOODWJqnhWyu4HNpH9nmPPyHCn6jp-VO4jnLfX7qKO00-BrbzbJnKTO3ythWzyTgg5OPU4rO89LqSW4vVmjM7q29SMMf4sjHfjpwMd66d_AkIhbZeOXAO390Ov51s6f4b0-0jDSQiecj55Jfn59uwFDAydB8OF2i1KYmBlwbeNR0Hq2fUf412AHoKYkYjRVQYUDAGamUcc0tgDFGcUv1pOlAADyMVes5oUdfMAznrjpVIDLUuCDQB0-2K8hC297ZxyZH3yBn25FaGtwWCaPFJBDDHNvAOzk9D3BINcSD0qaFiW_CgRa6moE5up_wDgP_oNTA8VAn_H1P8A8B_lQB574gtyNX1KED_WFiPxGaK0PF8Ai1uKY5CzRA591OP5YoqG7Gi1R58ATZYzJwTwfu_w9Pety5t3R5JHUJGIzguQN3yY4HU1klQLPGXzzwR8o6dD6-v4VoX277RMFHzcAevQVo1czZHYgjWmbZIw84hhFJsc7lwMGuhFn9mkO4LbOeqQ_vJT9Xasmzt86qHmin8oPEwMY-bKrxj3zW4qTMzAKYEzyBy5Puf_ANdZOajuVyt7DUEVsxKx7ZG998jfUmi40XXdRtxPZWD-Q2cNlclh35OT_Kus8GaBZandyCdyiQKZZHYgBl-p78g1sjUJpUGn6aDczktmcqFCLnjjp06ngegoUnLXoDXLoec6N4dliuHn1KJlnjcbVYg7iB944J6HOB0FdIBgc8Vt3GjLaQtLPNvbGWPbPrWMwFWhCjnAp4P1qIH3pwJJz-FMB-KMHoKB70uaABetSbc0xTUgPtQAoAxQGIozRn3pAO6cik6nim556VLGjO2EGTTAcgwA5XKg_ga37TT7S_tt5GCeGKn5lNRaJNA8n2W5CKTjZuGMn8e9aNzpItbjzdOnSG4IOYHYASAdQM_yotclsxr_AEK5so_OUCa27Sp0H19KoRIVY_kK6_StS0_T7KdJ7Zo7sFmZSDlyT0z1H49q5uVxJM74A3HOFGAKBkYJ9KgQ_wCmT_8AAP5VNJIkMTSOcIoyazbK7NxcTuRjJGB6CgCh4wtfO02K5ABaCTBPs3H88UVsXkCXtnNbP0kQqf6H86KTQ00ePOh-yoP3n8XUfL26f59K3L7ZDdOtvF5k7vgE9j6f559cdKWXR7uOOD7TCyNIwCQlSGwf4sdcV0S6fFYys8mZ7kjDk8DHovpTmC1KtrZC1iwELTAAyOOXZh6-g9PTFTKr3JwzfIv8_pU0cm8ZXGTnPGTyMU0IbaXzSSYzgSA849GrBUW3eRbqaWRdg4TyxwmMYFd61notr4btLi3upYVLK8jK_wA7kdVK9D3AHvn3riFQcFa1dG-xHUrcairPahuVBwPx9umfatzMusk-uyB2U21gpyiZ-Z_c-v16elZuqW8FmViRSG65rpddudN0S-uJrVEkupVUJGv3EA_iI6d-B6DsK4uaaSeZ5pXLuxyWNKwIibg8U5eFPapLeF7h9qAnHX2qR4Skhj-81AyIE4ozinmMqMEEEdjTMHNADxThwOlIik4qdIiSAByfWgCMUuwmrCwMsyxsDk46e9Wbmya12SEYjJAz6GmBnlMdantZzbzKx5Tv7ite-0v_AER54xlowGOO696xCPejcR1dxbWF1pf2mR9sajPnJ1X_AD6VLpt6bC-g_tJDIqqUjudp3bDg4OR2wO2R0rm7DUHsZGXAeGUbZYn-64_x9DXRaxr0V_ptvbQIRj5nLDkH0H9aBWKmv6gmo6g0kSqI1-VSBgsPU1kj65pc1lavfC3i8iIgSuOcfwj_AOvTAp6rffaJfJjP7pD1Hc-tM02TFww9VrMBJq3ZttukP1FBR0Aeiq6uKKRJv-MtPgHiXT4Npx5zzlsncxIAwT6DHFc_FaC-nYlgpYkdM4AoopRfur-urB7sz7i0FnfCNW3BuemPSp9g2kYoopjI7JirSwdVj27fYEdPwq6ODkUUUCILvO4Pnr2qDdyB60UUhnZ6JZRDSopOrTMxJ-h2ioNPtYrq_u5ZBzG-FHp1H9KKKlbkvqVLm2V9W8o_dJHGKp30SQ3JVFxgZooqikOsY993EueCf6VrXdrHDe2rDnzGww_ED-tFFIZa1a3SOySdeHjfj8j_AIVeuoUudOuUZcYh80H0IAP9aKKaJZV0W4nkmjDsjW6osLxFeX3EjOc8Y2-lYN7AtrfTwKcrHIyAnuAaKKbEiKJBIzE9FPQVaXAAoopFMSd_JheQDJVc4rjZXeaRpZG3OxyTRRTBAo_SpYOJkI67hRRQBrA9aKKKAP_Z
"""

def decode_base64_image(encoded_str,log_note='pass values to decode_base_64_jpg'): # - to +, _ to /
    """
    encoded_str: url safe base 64 jpg string --> image bytes string
    """
    try:
        image_bytes = base64.urlsafe_b64decode(encoded_str)
        image_obj = Image.open(BytesIO(image_bytes))
        image = Image.open(BytesIO(image_bytes)).convert('RGB')
        return image, image_obj
    except Exception as e:
        logging.error(f"{e} error encoding image at {log_note}")
        print(e)
        return None, None

def save_bytes_to_jpg(image_bytes, item_num=0,folder='Downloads/ml4g/image_data/',name='Fred'):
    filename = f'{folder}{name}_{item_num}.jpg'
    with open(filename, 'wb') as img_file:
        img_file.write(image_bytes)


image, image_bytes = encode_image(byte_str, transform_temp)


# Convert raw values to consistent feature vectors for the encoders:

In [57]:
from torch.utils.data import DataLoader

def collate_spatial(batch):
    nodes_ids, batch = zip(*batch)
    padded = nn.utils.rnn.pad_sequence(batch, batch_first=True)
    padded = padded.permute(0, 2, 1)
    return nodes_ids, padded

def collate_text(batch, min_size=20):
    if batch[0].size(0) < min_size:
        pad_num = min_size - batch[0].size(0)
        pads = torch.zeros(pad_num, dtype=torch.long, device=device)
        batch[0] = torch.cat([batch[0], pads], dim=0)
    return nn.utils.rnn.pad_sequence(batch, batch_first=True)

batch_size = 32

spatial_dataloader = DataLoader(spatial_dataset, batch_size=batch_size, collate_fn=collate_spatial)
text_dataloader_s = DataLoader(text_dataset_s, batch_size=batch_size, collate_fn=collate_text)
text_dataloader_m = DataLoader(text_dataset_m, batch_size=batch_size, collate_fn=collate_text)
text_dataloader_l = DataLoader(text_dataset_l, batch_size=1, collate_fn=collate_text) #it crashes every time it starts :'(
image_dataloader = DataLoader(image_dataset, batch_size=1)
datetime_dataloader = DataLoader(datetime_dataset, batch_size=batch_size)
year_dataloader = DataLoader(year_dataset, batch_size=batch_size)
numerical_dataloader = DataLoader(numerical_dataset, batch_size=batch_size)

In [8]:
# del polys_tupled, points_tupled,node_set, http_set, edge_set , string_set, string_set, string_set,image_set ,num_set ,poly_set ,datetime_set ,date_set ,year_set ,point_set ,text_edge_set ,text_edge_set ,text_edge_set ,image_edge_set ,num_edge_set ,spatial_edge_set ,temporal_edge_set ,encoder_map

# Multimodal Encoders

In [58]:
import torch.functional as f
import torch.nn as nn

# ]1] temporal conv
# Layer Filters Kernel Padding Pool
# 1 64 7 3 max(2/2)
# 2 64 7 3 max(2/2)
# 3 64 7 3 -
# 4 64 7 2 max(·)
# Layer Dimensions
# 5 512
# 6 128
# 7 128
class TextEncoder(nn.Module):
    def __init__(self, embed_dim, vocab_size=102, dropout=0.4, size_type='medium'): #filters = 64
        super(TextEncoder, self).__init__()
        self.size_type = size_type.lower()
        self.mlp_hidden_dim = 1024 if self.size_type == 'large' else 512 if self.size_type == 'medium' else 256
        self.tcnn_hidden_dim = 128 if self.size_type == 'large' else 64 if self.size_type == 'medium' else 32
        self.dilation_vals = (1,1,1) if self.size_type == 'small' else (2,4,8)

        
        self.embedding = nn.Embedding(vocab_size, embed_dim) #used embedding dict instead of ohe to save space
                                                            #sparse should probably work too
        self.conv1 = nn.Conv1d(
            embed_dim, self.tcnn_hidden_dim , kernel_size=7, padding=3, dilation=1)
        
        self.norm1 = nn.BatchNorm1d(self.tcnn_hidden_dim )
        self.drop1 = nn.Dropout(dropout)
        self.pool1 = nn.MaxPool1d(2,2)
        self.conv2 = nn.Conv1d(
            self.tcnn_hidden_dim , self.tcnn_hidden_dim , kernel_size=7, padding=3)

        self.norm2 = nn.BatchNorm1d(self.tcnn_hidden_dim )
        self.drop2 = nn.Dropout(dropout)
        self.pool2 = nn.MaxPool1d(2,2)
        self.conv3 = nn.Conv1d(
            self.tcnn_hidden_dim , self.tcnn_hidden_dim , kernel_size=7, padding=3)

        self.norm3 = nn.BatchNorm1d(self.tcnn_hidden_dim )
        self.drop3 = nn.Dropout(dropout)
        self.conv4 = nn.Conv1d(
            self.tcnn_hidden_dim , self.tcnn_hidden_dim , kernel_size=7, padding=2)#, dilation=self.dilation_vals[2])
        
        self.norm4 = nn.BatchNorm1d(self.tcnn_hidden_dim )
        self.drop4 = nn.Dropout(dropout)
        self.pool4 = nn.AdaptiveMaxPool1d(1)

        self.lin1 = nn.Linear(self.tcnn_hidden_dim, self.mlp_hidden_dim)
        self.lin2 = nn.Linear(self.mlp_hidden_dim, embed_dim)
        self.lin3 = nn.Linear(embed_dim , embed_dim)
        
        self.act = nn.ReLU()

    def forward(self, text):
        embedded = self.embedding(text).permute(0, 2, 1)
        residual = embedded
        #conv
        
        x = self.conv1(embedded)
        x = self.norm1(x)
        x = self.act(x)
        x = self.drop1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.act(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = self.act(x)
        x = self.drop3(x)
        x = self.conv4(x)
        x = self.norm4(x)
        x = self.act(x)
        x = self.drop4(x)
        # if residual.size(1) != x.size(1):
        #   residual = nn.utils.rnn.pad_sequence([residual, x], batch_first=True)
        x = self.pool4(x)
        x = x.view(x.size(0), -1)
        #ffnn
        x = self.lin1(x)
        x = self.lin2(x)
        # x += residual
        x = self.lin3(x)
        
        
            
        return x


# [2] MobileNet 
# Type / Stride Filter Shape Input Size
# Conv / s2 3 × 3 × 3 × 32 224 × 224 × 3
# Conv dw / s1 3 × 3 × 32 dw 112 × 112 × 32
# Conv / s1 1 × 1 × 32 × 64 112 × 112 × 32
# Conv dw / s2 3 × 3 × 64 dw 112 × 112 × 64
# Conv / s1 1 × 1 × 64 × 128 56 × 56 × 64
# Conv dw / s1 3 × 3 × 128 dw 56 × 56 × 128
# Conv / s1 1 × 1 × 128 × 128 56 × 56 × 128
# Conv dw / s2 3 × 3 × 128 dw 56 × 56 × 128
# Conv / s1 1 × 1 × 128 × 256 28 × 28 × 128
# Conv dw / s1 3 × 3 × 256 dw 28 × 28 × 256
# Conv / s1 1 × 1 × 256 × 256 28 × 28 × 256
# Conv dw / s2 3 × 3 × 256 dw 28 × 28 × 256
# Conv / s1 1 × 1 × 256 × 512 14 × 14 × 256
# 5×
# Conv dw / s1 3 × 3 × 512 dw 14 × 14 × 512
# Conv / s1 1 × 1 × 512 × 512 14 × 14 × 512
# Conv dw / s2 3 × 3 × 512 dw 14 × 14 × 512
# Conv / s1 1 × 1 × 512 × 1024 7 × 7 × 512
# Conv dw / s2 3 × 3 × 1024 dw 7 × 7 × 1024
# Conv / s1 1 × 1 × 1024 × 1024 7 × 7 × 1024
# Avg Pool / s1 Pool 7 × 7 7 × 7 × 1024
# FC / s1 1024 × 1000 1 × 1 × 1024
# Softmax / s1 Classifier 1 × 1 × 1000
# Table 2. Resource Per Layer Type
# Type Mult-Adds Parameters
# Conv 1 × 1 94.86% 74.59%
# Conv DW 3 × 3 3.06% 1.06%
# Conv 3 × 3 1.19% 0.02%
# Fully Connected 0.18% 24.33%

#idea: pass identity vector with batches to encoders (don't process, just pass back), use that to map embeddings to nodes later
class ImageEncoder(nn.Sequential):
    def __init__(self, embed_dim):
        super(ImageEncoder, self).__init__()
        

        self.image = MobileBlock(3, 32, normal_conv=True, stride=1, alpha=0.5)
        self.image2 = MobileBlock(32, 64, stride=1, alpha=0.5)
        self.image3 = MobileBlock(64, 128, stride=2, alpha=0.5)
        self.image4 = MobileBlock(128, 256, stride=1, alpha=0.5)
        self.image5 = MobileBlock(256, 256, stride=2, alpha=0.5)
        self.image6 = MobileBlock(256, 512, stride=1, alpha=0.5)
        self.image7 = MobileBlock(512, 512, stride=2, alpha=0.5)
        self.image8 = MobileBlock(512, 1024, stride=1, alpha=0.5)
        self.middle_conv = nn.Sequential(*[MobileBlock(
                1024, 1024, stride=1, alpha=0.5) for i in range(5)])
        
        self.image9 = MobileBlock(1024, 1024, stride=1, alpha=0.5)
        
        self.image10 = MobileBlock(1024, 2048, stride=2, alpha=0.5)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.lin1 = nn.Linear(1024,embed_dim)
        self.norm = nn.BatchNorm1d(embed_dim)
        self.act = nn.ReLU()
        # middle_conv = [MobileBlock() for i in range(5)]


    def forward(self, batch):
        x = self.image(batch)
        x = self.image2(x)
        x = self.image3(x)
        x = self.image4(x)
        x = self.image5(x)
        x = self.image6(x)
        x = self.image7(x)
        x = self.image8(x)
        x = self.middle_conv(x)
        x = self.image9(x)
        x = self.image10(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.lin1(x)
        x = self.norm(x)
        x = self.act(x)
        return x

class MiniImgEnc(nn.Sequential):
    def __init__(self, embed_dim):
        super(MiniImgEnc, self).__init__()
        self.image = MobileBlock(3, 32, normal_conv=True, stride=1, alpha=0.5)
        self.image2 = MobileBlock(32, 64, stride=1, alpha=0.5)
        self.image3 = MobileBlock(64, 128, stride=2, alpha=0.5)
        self.image4 = MobileBlock(128, 256, stride=1, alpha=0.5)
        self.image5 = MobileBlock(256, 512, stride=2, alpha=0.5)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.lin1 = nn.Linear(256,embed_dim)
        self.norm = nn.BatchNorm1d(embed_dim)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.image(x)
        x = self.image2(x)
        x = self.image3(x)
        x = self.image4(x)
        x = self.image5(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.lin1(x)
        # x = self.lin2(x)
        # x = self.norm(x)
        return self.act(x)
        

        

class MobileBlock(nn.Module):
    def __init__(self, in_channels, embedding_size, stride=1, normal_conv=False, alpha=0.5, dilation=1):
        super(MobileBlock, self).__init__()
        old_in_channels = in_channels
        in_channels = int(alpha * in_channels)
        embedding_size = int(alpha * embedding_size)
        # depthwise conv
        self.normal_conv = normal_conv
        if self.normal_conv:
            # in_channels = in_channels//2
            pass
        # 3x3x3x32
        self.conv = nn.Conv2d(old_in_channels, in_channels, 3, padding=1, stride=2, dilation=1)
        self.depthwise = nn.Conv2d(in_channels, in_channels, 3, stride=stride,
                                   padding=1, groups=in_channels)
        self.norm1 = nn.BatchNorm2d(in_channels)
        
        # pointwise conv
        self.pointwise = nn.Conv2d(in_channels, embedding_size, 1, stride=1, padding=0)
        self.norm2 = nn.BatchNorm2d(embedding_size)
        self.act = nn.ReLU(inplace=True)
        
    def forward(self, x):
        # print(self.conv.kernel_size)
        if self.normal_conv:
            x = self.conv(x)
        # print(x.size())
        x = self.depthwise(x)
        # print(x.size())
        x = self.norm1(x)
        x = self.act(x)
        
        x = self.pointwise(x)
        # print(x.size())
        x = self.norm2(x)
        return self.act(x)




# [3] temporal conv
# layer filters kernel padding pool
# 1 16 5 2 max(3/3)
# 2 32 5 2 -
# 3 64 5 2 avg(·)
# layer dimensions
# 4 512
# 5 128
# 6 128
class SpatialEncoder(nn.Module):
    def __init__(self, input_dim, embed_dim=128, dropout=0.2):
        super(SpatialEncoder, self).__init__()
        
        # temp cnn
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=16, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool1d(kernel_size=3, stride=3, ceil_mode=True)
        self.norm1 = nn.BatchNorm1d(16)
        self.drop1 = nn.Dropout(dropout)
        
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, padding=2)
        self.norm2 = nn.BatchNorm1d(32)
        self.drop2 = nn.Dropout(dropout)
        
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding=2)
        self.pool3 = nn.AdaptiveAvgPool1d(1)
        self.norm3 = nn.BatchNorm1d(64)
        self.drop3 = nn.Dropout(dropout)
        
        # dense
        self.lin1 = nn.Linear(64, 512)
        self.lin2 = nn.Linear(512, 128)
        self.lin3 = nn.Linear(128, embed_dim)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, spatial):
        # x = self.pad(spatial, batch_first=True)
        x = self.conv1(spatial)
        x = self.pool1(x)
        x = self.norm1(x)
        x = self.act(x)
        
        x = self.drop1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.act(x)
        x = self.drop2(x)
        
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.norm3(x)
        x = self.act(x)
        x = self.drop3(x)
        
        x = x.view(x.size(0), -1) #flatten
        
        x = self.lin1(x)
        x = self.act(x)
        x = self.dropout(x)
        
        x = self.lin2(x)
        x = self.act(x)
        x = self.dropout(x)
        
        x = self.lin3(x)
        x = self.act(x)
        x = self.dropout(x)
        return x
        

# [4] mlp h == in dim 1st col, out 2nd col
# XSD:gYear 6 2
# XSD:date 10 4
# XSD:dateTime 14 6
class TemporalEncoder(nn.Module):
    def __init__(self, input_dim, dropout=0.2):
        super(TemporalEncoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = input_dim
        self.out_dim = 2 if input_dim <= 6 else 4 if input_dim <= 9 else 6 #I messed something up but its very specific...
                                                                        #norm_cent returns a digit so idk if that should be 2?
                                                                        #there's no sin/cos, so it seems weird to encode it digitwise
        num_layers = 2
        self.mlp = nn.Sequential(*[nn.Linear(input_dim, input_dim), 
                                   nn.ReLU(),nn.BatchNorm1d(input_dim)]*num_layers)
        self.out = nn.Linear(input_dim, self.out_dim)

    def forward(self, x):
        x = self.mlp(x)
        return self.out(x)
    

#[5] one to one encoding for numerical
class NumericalEncoder(nn.Module):
    def __init__(self):
        super(NumericalEncoder, self).__init__()
        pass
    def forward(self, x):
        return x

# Combined

In [59]:
torch.cuda.empty_cache()


In [60]:
text_encoder_s, text_encoder_m, text_encoder_l = TextEncoder(128, size_type='s'), TextEncoder(128, size_type='m'), TextEncoder(128, size_type='m')
datetime_encoder = TemporalEncoder(9)
spatial_encoder = SpatialEncoder(5)
year_encoder = TemporalEncoder(5)
image_encoder = ImageEncoder(128)
numerical_encoder = NumericalEncoder()

class MultiModalEncoder(nn.Module):
    def __init__(self,
                 feature_matrix,
                 encoders_config):
        super(MultiModalEncoder, self).__init__()

        # self.dataloaders = [config['dataloader'] for config in encoders_config]
        self.encoders_config = encoders_config
        self.feature_matrix = feature_matrix
        self.column_map = {'text':(0,128),
                           'image':(129,257),
                           'numerical':(258,259),
                           'spatial':(260, 388),
                           'datetime':(389,393),
                           'year':(394,396)}
        self.x_indices = []#torch.zeros(8, device=device)
        self.y_indices = []
        self.values = []#torch.zeros((8,128), device=device)

        self.num_nodes = feature_matrix.size()[0]

    def forward(self):
        for config in self.encoders_config:
            name = config['name']
            encoder = config['encoder'].to(device)
            dataloader = config['dataloader']
            print(f"encoder: {name}")
            if 'image' not in name:
                for i, batch in enumerate(dataloader):
                    if 'text' in name:
                        cols = self.column_map['text']
                        node_ids = batch[:, 0]
                        features = batch[:, 1:]
                    else:
                        cols = self.column_map[name]
                        node_ids, features = batch
                        node_ids = torch.tensor(node_ids,device=device)
                    # I confused myself. passing the node_ids through the forward pass was completely unnecessary.
                    # change to sparse and use automatic column assignment base on thingy
                    try:
                        embeddings = encoder(features)
                    except ValueError: #if batch size is 0 or 1 when it's supposed to be more.
                        x,y = self.column_map[name]
                        output_dim = y-x
                        embeddings = torch.zeros(y-x, dtype=torch.float32,device=device).unsqueeze(0)
                    # embeddings = embeddings.to(device)
                    # self.feature_matrix[node_ids, cols[0]:cols[1]] = embeddings
                    # sparse assignment

                    # if embeddings.size(0) <= 1:
                    #     embeddings = embeddings.unsqueeze(0)
                    #     print('unsqueezed', embeddings.size())
                    batch_size = node_ids.size(0)
                    y_cols = torch.arange(cols[0], cols[1], device=device).unsqueeze(0).repeat(batch_size, 1)
                    # print(y_cols.size())
                    embedding_size = embeddings.size(1)
                    x_rows = node_ids.unsqueeze(1).repeat(1, embedding_size)
                    self.x_indices.append(x_rows)
                    self.y_indices.append(y_cols)
                    self.values.append(embeddings.view(-1))

                    
            print(f"encoder '{name}': processed {i}")

            # del dataset, dataloader
            try:
                
                x = torch.cat(self.x_indices).reshape(-1)
    
                # print(x.size())
                y = torch.cat(self.y_indices).reshape(-1)
                # print(y.size())
                indices = torch.stack([x, y], dim=0)
                # print(indices.size())
                values = torch.cat(self.values)
                # print(values.size())
                self.feature_matrix = torch.sparse_coo_tensor(
                    indices=indices,
                    values=values,
                    size=(self.num_nodes,396)
                    , device='cpu')
                torch.cuda.empty_cache()
            except Exception as e:
                print('magic has occurred stay spiffy\n', e)

        return self.feature_matrix

encoders_config = [{'name': 'text_s', 'encoder': text_encoder_s, 'dataloader': text_dataloader_s},
                   {'name': 'text_m', 'encoder': text_encoder_m, 'dataloader': text_dataloader_m},
                   {'name': 'text_l', 'encoder': text_encoder_l, 'dataloader': text_dataloader_l},
                   {'name': 'image', 'encoder': image_encoder, 'dataloader': image_dataloader},
                   {'name': 'numerical', 'encoder': numerical_encoder, 'dataloader': numerical_dataloader},
                   {'name': 'spatial', 'encoder': spatial_encoder, 'dataloader': spatial_dataloader},
                   {'name': 'datetime', 'encoder': datetime_encoder, 'dataloader': datetime_dataloader},
                   {'name': 'year', 'encoder': year_encoder, 'dataloader': year_dataloader},]


num_nodes = len(node_map)

embed_size = 128 + 128 + 1 + 128 + 4 + 2

feature_matrix = torch.zeros((num_nodes, embed_size), dtype=torch.float32, device=device).to_sparse()



multi_modal_encoder = MultiModalEncoder(
    feature_matrix=feature_matrix,
    encoders_config=encoders_config)

feature_matrix = multi_modal_encoder()

encoder: text_s
encoder 'text_s': processed 0
encoder: text_m
encoder 'text_m': processed 0
encoder: text_l
encoder 'text_l': processed 7
encoder: image
encoder 'image': processed 7
encoder: numerical
encoder 'numerical': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.
encoder: spatial
encoder 'spatial': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.
encoder: datetime
encoder 'datetime': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.
encoder: year
encoder 'year': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.


/tmp/ipykernel_381655/2935355595.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  node_ids = torch.tensor(node_ids,device=device)


# Connect to R-GCN

In [61]:
length = len(node_map.keys())

indices = torch.zeros((1),device=device)
indices = torch.arange(0, length, device=device).repeat(2, 1)

values = torch.ones(length, device=device)
identity = torch.sparse_coo_tensor(indices, values, (length, length), device=device)
print(identity.size())
print(feature_matrix.size())

torch.Size([4890, 4890])
torch.Size([4890, 396])


In [62]:
feature_matrix_cat = torch.cat((identity,feature_matrix),dim=1)
print(feature_matrix_cat.size(), 'Great success!')

torch.Size([4890, 5286]) Great success!


In [63]:
# feature_matrix

In [64]:


from typing import defaultdict
adjacency = defaultdict(torch.sparse_coo_tensor)
# inv_edge_map
indices_x = defaultdict(list)
indices_y = defaultdict(list)

values = defaultdict(list)
i=0
for s, p, o in graph:
    edge_id = inv_edge_map[p.identifier]
    # print(edge_id)
    s_id = inv_node_map[s.identifier]
    o_id = inv_node_map[o.identifier]
    
    indices_x[edge_id].append(s_id)
    indices_y[edge_id].append(o_id)
    values[edge_id].append(1)
    i += 1

graph_size = max(node_map.keys()) +1

for edge_id in indices_x.keys():
    ind_x, ind_y = torch.tensor(indices_x[edge_id]), torch.tensor(indices_y[edge_id])
    ind = torch.stack((ind_x, ind_y))
    val = values[edge_id]
    adjacency[edge_id] = torch.sparse_coo_tensor(indices=ind,values=val,size=(graph_size,graph_size),device=device)
    
i

5001

In [65]:
graph_size

4890

In [66]:
def normalize_adjacency(A, self=True): #gotta still add self-loops. See if it works without first.
    #it still crashes :(
    size = A.size()[0]
    A = A.coalesce()
    # A = torch.add(torch.eye(size,device=device).to_sparse(), A)
    eye_indices = torch.arange(0,size, device=device).repeat(2,1)
    eye_values = torch.ones(size, device=device)
    indices = torch.cat((A.indices(), eye_indices), dim=1)
    values = torch.cat((A.values(), eye_values), dim=0)
    A = torch.sparse_coo_tensor(indices, values, A.size(), device=device).coalesce()
    # print('to_dense()?')
    degree = torch.sparse.sum(A, dim=1).to_dense()
    # print('no')
    # print(degree.size())
    d_inv_sqrt = degree.pow(-0.5)
    D_inv_sqrt = torch.diag(d_inv_sqrt)
    # A = A.to_dense()
    normalized_values = values * d_inv_sqrt[indices[0]] * d_inv_sqrt[indices[1]]
    
    normalized_A = torch.sparse_coo_tensor(indices, normalized_values, A.size(), device=device).coalesce()
    # normalized_A = D_inv_sqrt @ A @ D_inv_sqrt
    return normalized_A


for i, key in enumerate(adjacency.keys()):
    A = adjacency[key]
    adjacency[key] = normalize_adjacency(A)
    del A
    print(f'Great success! {i}')
    k = key
adjacency[k]

Great success! 0
Great success! 1
Great success! 2
Great success! 3
Great success! 4
Great success! 5
Great success! 6
Great success! 7
Great success! 8
Great success! 9
Great success! 10
Great success! 11
Great success! 12
Great success! 13
Great success! 14
Great success! 15
Great success! 16
Great success! 17
Great success! 18
Great success! 19
Great success! 20


tensor(indices=tensor([[   0,    1,    2,  ..., 4887, 4888, 4889],
                       [   0,    1,    2,  ..., 4887, 4888, 4889]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(4890, 4890), nnz=4891, layout=torch.sparse_coo)

In [221]:
##save non-error version
def block_diag (weights):
    block_diag_matrix = torch.block_diag(*weights).to_sparse()
    return block_diag_matrix
    
class R_GCNLayer(nn.Module): 
    def __init__(self, x_dim, y_dim, num_relations, num_bases=30):
        super(R_GCNLayer, self).__init__()
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.num_relations = num_relations
        self.num_bases = num_bases

        gain = nn.init.calculate_gain('relu')
        self.W_b = nn.Parameter(torch.Tensor(self.num_bases, self.x_dim, self.y_dim))
        nn.init.kaiming_uniform_(self.W_b, a=gain)
        self.W_r = nn.Parameter(torch.Tensor(self.num_relations, self.num_bases))
        nn.init.kaiming_uniform_(self.W_r, a=gain)
        self.bias = nn.Parameter(torch.Tensor(self.y_dim))
        nn.init.zeros_(self.bias)

    def forward(self, A, X):
        aggregated = torch.zeros((X.size(0), self.y_dim), device=device)
        weights = torch.matmul(self.W_r, self.W_b.view(self.num_bases, -1))
        weights = weights.view(self.num_relations, self.x_dim, self.y_dim)        
        for r in range(self.num_relations):
            A_r = A[r]
            relation_weights = weights[r]
            weighted = torch.sparse.mm(A_r, X)
            # print(weighted.size(), relation_weights.size())
            weighted = torch.matmul(weighted, relation_weights)
            # print(weighted)
            aggregated += weighted
        
        aggregated += self.bias
        return aggregated

class R_GCN(nn.Module):
    def __init__(self, x_dim, h_dim, y_dim, max_k_hop, num_relations):
        super(R_GCN, self).__init__()
        self.max_k_hop =max_k_hop
        self.num_relations = num_relations
        self.x_dim = x_dim
        self.block_split = 2
        self.bases = 5
        self.gcns = nn.ModuleList([R_GCNLayer(x_dim, h_dim, num_relations, self.bases) for _ in range(max_k_hop)])
        self.final_r_gcn = R_GCNLayer(h_dim, y_dim,num_relations, self.bases) 
        self.act = nn.ReLU()
        self.drop = nn.Dropout(p=0.2)
        self.norm = nn.BatchNorm1d(y_dim)


    
        
    def forward(self, A, X):
        # if X.size(0) % 2 !=0:
        #     X = X[:-1,:]
        # outer layers
        device = next(self.parameters()).device
        for i in range(self.max_k_hop):
            H = self.gcns[i](A, X)
            H = self.act(H)
            H = self.drop(H)
        Y = self.final_r_gcn(A, H)
        out = self.act(Y)
        out = self.norm(out)
        return out#nn.LogSoftmax(Y)

#feature_matrix.size(0)+1
r_gcn = R_GCN(feature_matrix_cat.size(1), 32, num_classes, max_k_hop=2, num_relations=i+1).to(device)
out = r_gcn([normalize_adjacency(adjacency[relation_id]) for relation_id in adjacency.keys()], feature_matrix_cat)
out

tensor([[-0.0892, -0.0198, -0.2573,  0.1163, -0.1693],
        [-0.1562, -0.1180,  0.0649, -0.1702,  0.7330],
        [-0.1562,  1.4791,  0.1755,  0.6009,  0.1994],
        ...,
        [ 0.7725, -0.1799, -0.0812,  0.3648, -0.1693],
        [-0.1562, -0.1799,  0.5540,  0.0287, -0.1693],
        [-0.1562, -0.1799, -0.2716, -0.1702,  0.0227]],
       grad_fn=<NativeBatchNormBackward0>)

In [222]:
string_set_s

{"'Technau, Serg\\xe9'",
 '3231 BE',
 'Bicker Caarten, A.',
 'Collectie gebouwd',
 'Dukker, G.J.',
 'GEORGIUS, R.',
 'Galen, Paul van',
 'Heins, IJ.Th.',
 'Hoogendijk, C.',
 'Roderburg, Kris',
 'Scanlan, G.',
 'Schollen, A.H.C.',
 'Schollen, T.',
 'Terpstra, Frank',
 'Voorde", exterieur',
 'kleur negatief',
 'rechter zijgevel',
 'voor- en zijgevel',
 'zwart wit negatief'}

In [223]:
feature_matrix

tensor(indices=tensor([[4320, 4320, 4320,  ..., 3630, 3630, 3630],
                       [   0,    1,    2,  ...,  125,  126,  127]]),
       values=tensor([ 0.3352, -0.1468, -0.1269,  ..., -0.1078, -0.4828,
                       0.2081]),
       size=(4890, 396), nnz=4864, layout=torch.sparse_coo,
       grad_fn=<SparseCooTensorWithDimsAndTensorsBackward0>)

In [224]:
6338/2

3169.0

In [225]:
num_nodes = length

# Why are these zeros...?           ^

In [226]:
# A_list = [normalize_adjacency(adjacency[relation_id]) for relation_id in adjacency.keys()]

In [227]:
class MR_GCN(nn.Module):
    def __init__(self, A_list, eye, num_nodes, rgcn_inp_dim, embed_size=396, num_classes=5):
        super(MR_GCN, self).__init__()
        
        self.A_list = A_list
        self.embed_size = embed_size
        self.embedding_matrix = torch.zeros((num_nodes, embed_size), dtype=torch.float32, device=device).to_sparse()
        self.MME= MultiModalEncoder(feature_matrix=self.embedding_matrix, encoders_config=encoders_config).to(device)
        self.graph_dim = A_list[0].size(0)
        self.R_GCN = R_GCN(rgcn_inp_dim, 128, num_classes, max_k_hop=2, num_relations=len(A_list)).to(device)
        self.eye = eye
        self.feature_matrix = []


    def forward(self):
        X = self.MME()
        X = torch.cat((self.eye,X), dim=1)
        X = self.R_GCN(self.A_list, X)
        return X
mr_gcn = MR_GCN([normalize_adjacency(adjacency[relation_id]) for relation_id in adjacency.keys()], identity, num_nodes, feature_matrix_cat.size(1))

In [228]:
mr_gcn()

encoder: text_s
encoder 'text_s': processed 0
encoder: text_m
encoder 'text_m': processed 0
encoder: text_l
encoder 'text_l': processed 7
encoder: image
encoder 'image': processed 7
encoder: numerical
encoder 'numerical': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.
encoder: spatial
encoder 'spatial': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.
encoder: datetime
encoder 'datetime': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.
encoder: year
encoder 'year': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.


/tmp/ipykernel_381655/2935355595.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  node_ids = torch.tensor(node_ids,device=device)


tensor([[-0.1803, -0.6593, -0.4005, -0.0929, -0.3876],
        [ 0.1452,  0.6557,  0.8066, -0.0929, -0.4942],
        [-0.1803, -0.7118,  1.3599, -0.0929,  0.2618],
        ...,
        [ 0.1165, -0.7118,  0.3027, -0.0929, -0.4942],
        [-0.1803,  0.4252, -0.4005, -0.0929,  0.1609],
        [ 1.0363, -0.4621,  0.2865, -0.0929, -0.4942]],
       grad_fn=<NativeBatchNormBackward0>)

In [229]:
labels = list(train_label_set|val_label_set|test_label_set)
num_classes = len(labels)
ohe = torch.eye(num_classes)
label_to_ohe = {label:ohe[i] for i, label in enumerate(labels)}
label_to_num = {label:i for i, label in enumerate(labels)}

labels = torch.full((num_nodes,), -1, dtype=torch.long, device=device)

train_labels = labels
for node_id, label in train_node_label_map.items():
    train_labels[node_id] = label_to_num[label]

train_mask = labels>=0

test_labels = labels
for node_id, label in test_node_label_map.items():
    test_labels[node_id] = label_to_num[label]

test_mask = labels>=0

val_labels = labels
for node_id, label in val_node_label_map.items():
    val_labels[node_id] = label_to_num[label]

val_mask = labels>=0

In [230]:
num_classes

5

In [231]:
labels[train_mask].size()

torch.Size([40])

In [232]:
outputs[train_mask].size()

torch.Size([40, 5])

In [233]:
criterion = nn.CrossEntropyLoss()
params = list(mr_gcn.parameters())
optimizer = torch.optim.Adam(params,lr=0.01)

num_epochs = 600
mr_gcn = MR_GCN([normalize_adjacency(adjacency[relation_id]) for relation_id in adjacency.keys()], identity, num_nodes, feature_matrix_cat.size(1), num_classes=num_classes)

for epoch in range(num_epochs):
    mr_gcn.train()
    optimizer.zero_grad()
    
    outputs = mr_gcn()
    print(outputs.size())
    loss = criterion(outputs[train_mask], labels[train_mask])
    
    loss.backward()
    optimizer.step()
 

encoder: text_s
encoder 'text_s': processed 0
encoder: text_m
encoder 'text_m': processed 0
encoder: text_l
encoder 'text_l': processed 7
encoder: image
encoder 'image': processed 7
encoder: numerical
encoder 'numerical': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.
encoder: spatial
encoder 'spatial': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.
encoder: datetime
encoder 'datetime': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.
encoder: year
encoder 'year': processed 0
magic has occurred stay spiffy
 Sizes of tensors must match except in dimension 0. Expected size 128 but got size 1 for tensor number 10 in the list.
torch.Size([4890, 5])


/tmp/ipykernel_381655/2935355595.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  node_ids = torch.tensor(node_ids,device=device)


RuntimeError: mat2_.is_sparse() INTERNAL ASSERT FAILED at "/pytorch/aten/src/ATen/native/sparse/SparseMatMul.cpp":267, please report a bug to PyTorch. 

In [ ]:
labels[train_mask]

In [234]:
val_mask[335]

tensor(True)

In [102]:
test_node_label_map

defaultdict(list,
            {3432: 'https://data.labs.pdok.nl/rce/def/agrarischGebouw',
             4364: 'https://data.labs.pdok.nl/rce/def/agrarischGebouw',
             2458: 'https://data.labs.pdok.nl/rce/def/kerk',
             3525: 'https://data.labs.pdok.nl/rce/def/agrarischGebouw',
             1460: 'https://data.labs.pdok.nl/rce/def/agrarischGebouw',
             3475: 'https://data.labs.pdok.nl/rce/def/kasteelLandhuis',
             3274: 'https://data.labs.pdok.nl/rce/def/molen',
             4393: 'https://data.labs.pdok.nl/rce/def/kasteelLandhuis',
             4036: 'https://data.labs.pdok.nl/rce/def/kerk',
             1597: 'https://data.labs.pdok.nl/rce/def/woonhuis',
             2776: 'https://data.labs.pdok.nl/rce/def/kasteelLandhuis',
             422: 'https://data.labs.pdok.nl/rce/def/agrarischGebouw',
             4211: 'https://data.labs.pdok.nl/rce/def/agrarischGebouw',
             335: 'https://data.labs.pdok.nl/rce/def/woonhuis'})

In [87]:
train_mask[]

tensor(False)

In [82]:
label_to_ohe

{'https://data.labs.pdok.nl/rce/def/woonhuis': tensor([1., 0., 0., 0., 0.]),
 'https://data.labs.pdok.nl/rce/def/agrarischGebouw': tensor([0., 1., 0., 0., 0.]),
 'https://data.labs.pdok.nl/rce/def/molen': tensor([0., 0., 1., 0., 0.]),
 'https://data.labs.pdok.nl/rce/def/kasteelLandhuis': tensor([0., 0., 0., 1., 0.]),
 'https://data.labs.pdok.nl/rce/def/kerk': tensor([0., 0., 0., 0., 1.])}